In [1]:
# python imports
import sys
sys.path.append('/usr/local/bin/chromedriver')
sys.path.append('/opt/miniconda3/envs/python3/bin/geckodriver')
sys.path

import os
import math
import json
import pickle
import itertools
from pathlib import Path
from contextlib import redirect_stdout

# third lib stuff
import numpy as np
from pycocotools.cocoeval import COCOeval
from tqdm.notebook import tqdm
from prettytable import PrettyTable

# plotting stuff
from bokeh.io import output_notebook, show, export_svg
from bokeh.models import ColumnDataSource, FixedTicker, Legend, LegendItem, Div, Title, LinearColorMapper, ColorBar
from bokeh.models.ranges import FactorRange
from bokeh.palettes import Category20, Category20b, Category20c, magma, Turbo256
from bokeh.plotting import figure
from bokeh.layouts  import gridplot, column, row
from bokeh.transform import transform
output_notebook()

Loading BokehJS ...

In [2]:
ROOT = Path("./").resolve()

DETECTRON = ROOT / "detectron"
# https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md
BASELINES_IN_PRE = {
    "mask_rcnn_R_50_C4_1x":   [36.8, 32.2],
    "mask_rcnn_R_50_DC5_1x":  [38.3, 34.2],
    "mask_rcnn_R_50_FPN_1x":  [38.6, 35.2],
    "mask_rcnn_R_50_C4_3x":   [39.8, 34.4],
    "mask_rcnn_R_50_DC5_3x":  [40.0, 35.9],
    "mask_rcnn_R_50_FPN_3x":  [41.0, 37.2],
    "mask_rcnn_R_101_C4_3x":  [42.6, 36.7],
    "mask_rcnn_R_101_DC5_3x": [41.9, 37.3],
    "mask_rcnn_R_101_FPN_3x": [42.9, 38.6],
    "mask_rcnn_X_101_32x8d_FPN_3x": [44.3, 39.5]
    }
BASELINES_NO_PRE = {
    "mask_rcnn_R_50_FPN_100ep_LSJ":  [44.6, 40.3],
    "mask_rcnn_R_50_FPN_200ep_LSJ":  [46.3, 41.7],
    "mask_rcnn_R_50_FPN_400ep_LSJ":  [47.4, 42.5],
    "mask_rcnn_R_101_FPN_100ep_LSJ": [46.4, 41.6],
    "mask_rcnn_R_101_FPN_200ep_LSJ": [48.0, 43.1],
    "mask_rcnn_R_101_FPN_400ep_LSJ": [48.9, 43.7],
    "mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ": [46.0, 41.3],
    "mask_rcnn_regnetx_4gf_dds_FPN_200ep_LSJ": [48.1, 43.1],
    "mask_rcnn_regnetx_4gf_dds_FPN_400ep_LSJ": [48.6, 43.5],
    "mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ": [46.1, 41.6],
    "mask_rcnn_regnety_4gf_dds_FPN_200ep_LSJ": [47.8, 43.0],
    "mask_rcnn_regnety_4gf_dds_FPN_400ep_LSJ": [48.2, 43.3]
    }
OTHER_SETTINGS = {
    "cascade_mask_rcnn_R_50_FPN_1x": [42.1, 36.4],
    "cascade_mask_rcnn_R_50_FPN_3x": [44.3, 38.5],
    "mask_rcnn_R_50_FPN_1x_dconv_c3-c5": [41.5, 37.5],
    "mask_rcnn_R_50_FPN_3x_dconv_c3-c5": [42.7, 38.5],
    "panoptic_fpn_R_101_dconv_cascade_gn_3x": [47.4, 41.3],
    "cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv": [50.2, 44.0]
    }
# https://github.com/HobbitLong/PyContrast/tree/master/pycontrast/detection
SELF_SUPERVISED = {
    "R_50_FPN_Random_6x":     [42.7, 38.6],
    "R_50_FPN_Supervised_6x": [42.6, 38.5],
    "R_50_FPN_InstDis_1x":    [38.8, 35.2],
    "R_50_FPN_InstDis_2x":    [41.3, 37.3],
    "R_50_FPN_PIRL_1x":       [38.6, 35.1],
    "R_50_FPN_PIRL_2x":       [41.2, 37.4],
    "R_50_FPN_MoCo_v1_1x":    [39.4, 35.6],
    "R_50_FPN_MoCo_v1_2x":    [41.7, 37.5],
    "R_50_FPN_MoCo_v2_2x":    [41.7, 37.6],
    "R_50_FPN_InfoMin_1x":    [40.6, 36.7],
    "R_50_FPN_InfoMin_2x":    [42.5, 38.4],
    "R_50_FPN_InfoMin_6x":    [43.6, 39.2]
    }

# https://gist.github.com/szagoruyko/9c9ebb8455610958f7deaa27845d7918
DETR = ROOT / "detr"
DETR_PANOPTIC = {
    "detr_r50":      [38.8, 31.1],
    "detr_r50_dc5": [40.2, 31.9],
    "detr_r101":     [40.1, 33.0]
}

# https://github.com/AlexeyAB/darknet
# no scores reported for val2017, we checked test-dev with coco server though
DARKNET = ROOT / "darknet"
YOLO = {
    "yolov3" :    [-1, -1],
    "yolov4" :    [-1, -1],
    "yolov4-csp": [-1, -1],
    "yolov4-p5":  [-1, -1],
    "yolov4-p6":  [-1, -1]
}

# https://github.com/SwinTransformer/Swin-Transformer-Object-Detection
SWIN = ROOT / "swin"
SWIN_MODELS = {
    "mask_rcnn_swin-T_1x" : [43.7, 39.8],
    "mask_rcnn_swin-T_3x" : [46.0, 41.6],
    "mask_rcnn_swin-S_3x" : [48.5, 43.3],  
    "cascade_mask_rcnn_swin-T_1x" : [48.1, 41.7],
    "cascade_mask_rcnn_swin-T_3x" : [50.4, 43.7],  
    "cascade_mask_rcnn_swin-S_3x" : [52.0, 45.0],
    "cascade_mask_rcnn_swin-B_3x" : [51.9, 45.0]
}

# https://github.com/easton-cau/SOTR
# no scores reported for val2017, we checked test-dev with coco server though
SOTR = ROOT / "sotr"
SOTR_MODELS = {
    "sotr_R101" :    [-1, -1],
    "sotr_R101_dcn": [-1, -1]
}

# https://github.com/aim-uofa/AdelaiDet/tree/master/configs/SOLOv2
# no scores reported for val2017, we checked test-dev with coco server though
SOLO = ROOT / "solov2"
SOLO_MODELS = {
    "solov2_R50_3x" : [-1, -1],
    "solov2_R101_3x": [-1, -1]
}

# https://github.com/dbolya/yolact
# no scores reported for val2017, we checked test-dev with coco server though
YOLACT = ROOT / "yolact"
YOLACT_MODELS = {
    "yolact-R50-FPN" :  [-1, -1],
    "yolact-R101-FPN" : [-1, -1],
    "yolact-plus-R50-FPN":  [-1, -1],
    "yolact-plus-R101-FPN": [-1, -1]
}
BMASK = ROOT / "bmask"
BMASK_MODELS = {
    "bmask_rcnn_r50_1x" : [-1, 36.6],
    "bmask_rcnn_r101_1x": [-1, 38.0],
    "cascade_bmask_rcnn_r50_1x" : [-1, 37.5],
    "cascade_bmask_rcnn_r101_1x" : [-1, 39.1]
}
BCNET = ROOT / "bcnet"
BCNET_MODELS = {
    "fcos_imprv_R_101_FPN" : [-1, 39.6]
}

In [3]:
# COCO EVAL format https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/cocoeval.py

# precision  - [TxRxKxAxM] 
# recall     - [TxKxAxM]
# T: thresholds
# R: recall thresholds
# K: categories
# A: areas 
# M: max detections

def printCocoEvalParams(eval_object):
    params = eval_object['params']
    print(f"IOU Type: {params.iouType}")
    print(f"Counts: {eval_object['counts']}")      
    print(f"T ( {len(params.iouThrs)}): {params.iouThrs}")
    print(f"R ({len(params.recThrs)}): {min(params.recThrs)}, ..., {max(params.recThrs)}")
    print(f"K ( {len(params.catIds)}): {min(params.catIds)}, ..., {max(params.catIds)} ")
    print(f"A (  {len(params.areaRngLbl)}): {params.areaRngLbl}")
    print(f"M (  {len(params.maxDets)}): {params.maxDets}")

# custom summarize method:
# runs summarize() on a loaded eval dict {} and returns stats instead of printing them
def COCOeval_summarize(loaded_eval_dict):
    eval_proxy = COCOeval()
    eval_proxy.eval   = loaded_eval_dict
    eval_proxy.params = loaded_eval_dict['params']
    eval_proxy.summarize()
    return eval_proxy.stats

## Load detection results

In [4]:
COCO = "coco/val2017"
STYLIZED_COCO            = "stylized_coco"
STYLIZED_COCO_OBJECTS    = "stylized_coco_objects"
STYLIZED_COCO_BACKGROUND = "stylized_coco_background"
COCO_BLUR = "coco_blur/val2017"
COCO_SAP  = "coco_sap/val2017"
SAP   = "sap_coco"
SAP_O = "sap_objects"
SAP_B = "sap_background"
STYLE_VERSIONS = ["1"]

ALPHAS_fs   = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
ALPHAS_ps   =      [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]      # 0.0=coco, 1.0 same as fs 1.0
BLEND_MODES = {"pixel_space": ALPHAS_ps, "feature_space": ALPHAS_fs}

EVAL_BBOX  = "coco_eval_bbox.pkl"          # coco eval object 'bbox'
EVAL_SEGM  = "coco_eval_segm.pkl"          # coco eval object 'segm'
#IOU_TYPES  = {'bbox': EVAL_BBOX, 'segm': EVAL_SEGM}
IOU_TYPES  = {'segm': EVAL_SEGM}

In [5]:
RESULTS = {}
MODELS  = {}
MODELS.update(BASELINES_IN_PRE)
MODELS.update(BASELINES_NO_PRE)
MODELS.update(SELF_SUPERVISED)
MODELS.update(OTHER_SETTINGS)
MODELS.update(DETR_PANOPTIC)
MODELS.update(YOLO)
MODELS.update(SWIN_MODELS)
MODELS.update(SOTR_MODELS)
MODELS.update(SOLO_MODELS)
MODELS.update(YOLACT_MODELS)
MODELS.update(BMASK_MODELS)
MODELS.update(BCNET_MODELS)

# load results
for model in tqdm(list(MODELS.keys())):    
    RESULTS[model] = {}

    # add results from coco and blurred coco
    RESULTS[model]['coco'] = {}
    RESULTS[model]['coco_blur'] = {}
    for iou_type, eval_file in IOU_TYPES.items():
        if iou_type == 'segm' and model in YOLO.keys(): #continue
            iou_type  = 'bbox'
            eval_file = "coco_eval_bbox.pkl"  
        # set path
        if 'detr' in model:                 PATH = DETR
        elif 'yolo' in model:               PATH = DARKNET
        elif model in SWIN_MODELS.keys():   PATH = SWIN
        elif model in SOTR_MODELS.keys():   PATH = SOTR
        elif model in SOLO_MODELS.keys():   PATH = SOLO
        elif model in YOLACT_MODELS.keys(): PATH = YOLACT
        elif model in BMASK_MODELS.keys():  PATH = BMASK
        elif model in BCNET_MODELS.keys():  PATH = BCNET
        else: 
            PATH = DETECTRON

        with open(PATH / model / COCO / eval_file, 'rb') as f:
            eval_dict = pickle.load(f)
            with redirect_stdout(None): # printing is slow
                RESULTS[model]['coco'][iou_type] = COCOeval_summarize(eval_dict)
             
        if os.path.isdir(PATH / model / COCO_BLUR):
            print('Found BLUR for:', model)
            with open(PATH / model / COCO_BLUR / eval_file, 'rb') as f:
                eval_dict = pickle.load(f)
                with redirect_stdout(None): # printing is slow
                    RESULTS[model]['coco_blur'][iou_type] = COCOeval_summarize(eval_dict)

    # add results from stylized coco, objects, background
    for dataset in [STYLIZED_COCO, STYLIZED_COCO_OBJECTS, STYLIZED_COCO_BACKGROUND]:
        RESULTS[model][dataset] = {}
        for style in STYLE_VERSIONS: # 1
            RESULTS[model][dataset][style] = {}
            for mode, alpha_values in BLEND_MODES.items(): # feature space, pixel_space
                RESULTS[model][dataset][style][mode] = {}
                for alpha in alpha_values:
                    RESULTS[model][dataset][style][mode][alpha] = {}
                    for iou_type, eval_file in IOU_TYPES.items():
                        if iou_type == 'segm' and model in YOLO.keys(): #continue
                            iou_type = 'bbox'
                            eval_file = "coco_eval_bbox.pkl"  
                        if 'detr' in model:                 PATH = DETR
                        elif 'yolo' in model:               PATH = DARKNET
                        elif model in SWIN_MODELS.keys():   PATH = SWIN
                        elif model in SOTR_MODELS.keys():   PATH = SOTR
                        elif model in SOLO_MODELS.keys():   PATH = SOLO
                        elif model in YOLACT_MODELS.keys(): PATH = YOLACT
                        elif model in BMASK_MODELS.keys():  PATH = BMASK
                        elif model in BCNET_MODELS.keys():  PATH = BCNET
                        else:
                            PATH = DETECTRON

                        with open(PATH / model / dataset / style / mode / str(alpha) / eval_file, 'rb') as f:
                            eval_dict = pickle.load(f)
                            with redirect_stdout(None): # printing is slow
                                RESULTS[model][dataset][style][mode][alpha][iou_type] = COCOeval_summarize(eval_dict)

    if os.path.isdir(PATH / model / COCO_SAP):
        print(f'Found SAP for {model}')
        for dataset in [COCO_SAP, SAP, SAP_O, SAP_B]:    
            RESULTS[model][dataset] = {}
            for iou_type, eval_file in IOU_TYPES.items():
                if iou_type == 'bbox': continue
                if   model in SOTR_MODELS.keys():     PATH = SOTR
                elif model in SOLO_MODELS.keys():   PATH = SOLO
                elif model in YOLACT_MODELS.keys(): PATH = YOLACT
                elif model in SWIN_MODELS.keys():   PATH = SWIN
                else:
                    PATH = DETECTRON

                with open(PATH / model / dataset / eval_file, 'rb') as f:
                    eval_dict = pickle.load(f)
                    with redirect_stdout(None): # printing is slow
                        RESULTS[model][dataset][iou_type] = COCOeval_summarize(eval_dict)


  0%|          | 0/68 [00:00<?, ?it/s]

Found BLUR for: mask_rcnn_R_50_DC5_3x
Found SAP for mask_rcnn_R_50_DC5_3x
Found BLUR for: mask_rcnn_R_50_FPN_3x
Found SAP for mask_rcnn_R_50_FPN_3x
Found BLUR for: mask_rcnn_R_101_DC5_3x
Found SAP for mask_rcnn_R_101_DC5_3x
Found BLUR for: mask_rcnn_R_101_FPN_3x
Found SAP for mask_rcnn_R_101_FPN_3x
Found BLUR for: mask_rcnn_R_50_FPN_400ep_LSJ
Found SAP for mask_rcnn_R_50_FPN_400ep_LSJ
Found BLUR for: mask_rcnn_R_50_FPN_3x_dconv_c3-c5
Found SAP for mask_rcnn_R_50_FPN_3x_dconv_c3-c5
Found BLUR for: mask_rcnn_swin-T_3x
Found SAP for mask_rcnn_swin-T_3x
Found BLUR for: mask_rcnn_swin-S_3x
Found SAP for mask_rcnn_swin-S_3x
Found BLUR for: cascade_mask_rcnn_swin-B_3x
Found SAP for cascade_mask_rcnn_swin-B_3x
Found BLUR for: sotr_R101
Found SAP for sotr_R101
Found BLUR for: sotr_R101_dcn
Found SAP for sotr_R101_dcn
Found BLUR for: solov2_R50_3x
Found SAP for solov2_R50_3x
Found BLUR for: solov2_R101_3x
Found SAP for solov2_R101_3x
Found BLUR for: yolact-R50-FPN
Found SAP for yolact-R50-FPN
Fo

In [6]:
print(RESULTS["mask_rcnn_R_50_DC5_3x"].keys())
#print(RESULTS["mask_rcnn_R_50_DC5_3x"]['coco_blur'])

dict_keys(['coco', 'coco_blur', 'stylized_coco', 'stylized_coco_objects', 'stylized_coco_background', 'coco_sap/val2017', 'sap_coco', 'sap_objects', 'sap_background'])


In [7]:
STATS = {
    "AP"   :  0, #  0: Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]
    "AP50" :  1, #  1: Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ]
    "AP75" :  2, #  2: Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ]
    "APs"  :  3, #  3: Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]
    "APm"  :  4, #  4: Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]
    "APl"  :  5, #  5: Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]
    "AR1"  :  6, #  6: Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]
    "AR10" :  7, #  7: Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]
    "AR100":  8, #  8: Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]
    "ARs"  :  9, #  9: Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]
    "ARm"  : 10, # 10: Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]
    "ARl"  : 11  # 11: Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]
}

# sort by statistic for easy plotting, alphas are concatenated
SUMMARY = {}
for model in list(MODELS.keys()):    
    SUMMARY[model] = {}
    
    # Blurred coco
    if RESULTS[model].get('coco_blur'):
        print('Adding Blur for:', model)
        SUMMARY[model]['coco_blur'] = {}
        SUMMARY[model]['coco_blur']['segm'] = {}
        
        # new sort by statistics now
        for stat_key, stat_idx in STATS.items():
            SUMMARY[model]['coco_blur']['segm'][stat_key] = []
            
            # add original coco for comparison
            s = RESULTS[model]['coco']['segm'][stat_idx]
            SUMMARY[model]['coco_blur']['segm'][stat_key].append(s)
            
            s = RESULTS[model]['coco_blur']['segm'][stat_idx]
            SUMMARY[model]['coco_blur']['segm'][stat_key].append(s)
    
    #for stat_key, stat_idx in STATS.items():
        
    if RESULTS[model].get('coco_sap/val2017'):
        print('Adding SAP for:', model)
        
        for dataset in [SAP, SAP_O, SAP_B]:
        
            SUMMARY[model][dataset] = {}
            SUMMARY[model][dataset]['segm'] = {}

            # new sort by statistics now
            for stat_key, stat_idx in STATS.items():
                SUMMARY[model][dataset]['segm'][stat_key] = []
                # add coco (alpha=0 in pixel space)
                s = RESULTS[model]['coco']['segm'][stat_idx]
                SUMMARY[model][dataset]['segm'][stat_key].append(s)

                s = RESULTS[model]['coco_sap/val2017']['segm'][stat_idx]
                SUMMARY[model][dataset]['segm'][stat_key].append(s)

                s = RESULTS[model][dataset]['segm'][stat_idx]
                SUMMARY[model][dataset]['segm'][stat_key].append(s)
    
    for dataset in [STYLIZED_COCO, STYLIZED_COCO_OBJECTS, STYLIZED_COCO_BACKGROUND]:
        SUMMARY[model][dataset] = {}
        for style in STYLE_VERSIONS: # 1
            SUMMARY[model][dataset][style] = {}
            for mode, alpha_values in BLEND_MODES.items(): # feature space, pixel_space
                SUMMARY[model][dataset][style][mode] = {}
                for iou_type in IOU_TYPES.keys():
                    if iou_type == 'segm' and model in YOLO.keys(): #continue
                        iou_type = 'bbox'
                    SUMMARY[model][dataset][style][mode][iou_type] = {}

                    # new sort by statistics now
                    for stat_key, stat_idx in STATS.items():
                        SUMMARY[model][dataset][style][mode][iou_type][stat_key] = []
                        if mode == 'feature_space':
                            for alpha in alpha_values:
                                s = RESULTS[model][dataset][style][mode][alpha][iou_type][stat_idx]
                                SUMMARY[model][dataset][style][mode][iou_type][stat_key].append(s)
                        if mode == 'pixel_space':    
                            # add coco (alpha=0 in pixel space)
                            s = RESULTS[model]['coco'][iou_type][stat_idx]
                            SUMMARY[model][dataset][style][mode][iou_type][stat_key].append(s)

                            # add [0.1, ..., 0.9]
                            for alpha in alpha_values:
                                s = RESULTS[model][dataset][style][mode][alpha][iou_type][stat_idx]
                                SUMMARY[model][dataset][style][mode][iou_type][stat_key].append(s)

                            # add stylized coco, object, background (alpha=1 to pixel space list)
                            s = RESULTS[model][dataset][style]['feature_space'][1.0][iou_type][stat_idx]
                            SUMMARY[model][dataset][style][mode][iou_type][stat_key].append(s)


Adding Blur for: mask_rcnn_R_50_DC5_3x
Adding SAP for: mask_rcnn_R_50_DC5_3x
Adding Blur for: mask_rcnn_R_50_FPN_3x
Adding SAP for: mask_rcnn_R_50_FPN_3x
Adding Blur for: mask_rcnn_R_101_DC5_3x
Adding SAP for: mask_rcnn_R_101_DC5_3x
Adding Blur for: mask_rcnn_R_101_FPN_3x
Adding SAP for: mask_rcnn_R_101_FPN_3x
Adding Blur for: mask_rcnn_R_50_FPN_400ep_LSJ
Adding SAP for: mask_rcnn_R_50_FPN_400ep_LSJ
Adding Blur for: mask_rcnn_R_50_FPN_3x_dconv_c3-c5
Adding SAP for: mask_rcnn_R_50_FPN_3x_dconv_c3-c5
Adding Blur for: mask_rcnn_swin-T_3x
Adding SAP for: mask_rcnn_swin-T_3x
Adding Blur for: mask_rcnn_swin-S_3x
Adding SAP for: mask_rcnn_swin-S_3x
Adding Blur for: cascade_mask_rcnn_swin-B_3x
Adding SAP for: cascade_mask_rcnn_swin-B_3x
Adding Blur for: sotr_R101
Adding SAP for: sotr_R101
Adding Blur for: sotr_R101_dcn
Adding SAP for: sotr_R101_dcn
Adding Blur for: solov2_R50_3x
Adding SAP for: solov2_R50_3x
Adding Blur for: solov2_R101_3x
Adding SAP for: solov2_R101_3x
Adding Blur for: yolact

In [12]:
for stat in ['AP', 'APs', 'APm', 'APl']:   
    print(stat)
    for k, v in SUMMARY.items():
        if v.get('sap_coco'):
            r = []
            for d in ['sap_coco', 'sap_objects', 'sap_background']:
                c, c_t, s = v[d]['segm'][stat]
                c, c_t, s = np.round(c*100,2), np.round(c_t*100,2), np.round(s*100,2)
                #r.append(np.round((c_t/c)*100,2))
                r.append(np.round((s/c)*100,2))
            print(k,r)
    print()
    

AP
mask_rcnn_R_50_DC5_3x [12.61, 29.37, 85.94]
mask_rcnn_R_50_FPN_3x [17.46, 32.02, 86.33]
mask_rcnn_R_101_DC5_3x [15.02, 28.52, 88.4]
mask_rcnn_R_101_FPN_3x [17.45, 30.13, 86.82]
mask_rcnn_R_50_FPN_400ep_LSJ [23.27, 38.03, 86.56]
mask_rcnn_R_50_FPN_3x_dconv_c3-c5 [19.76, 34.98, 89.77]
mask_rcnn_swin-T_3x [30.42, 39.86, 92.0]
mask_rcnn_swin-S_3x [36.63, 45.65, 92.18]
cascade_mask_rcnn_swin-B_3x [32.01, 41.29, 91.6]
sotr_R101 [14.77, 31.16, 86.79]
sotr_R101_dcn [25.38, 40.14, 92.78]
solov2_R50_3x [16.32, 32.11, 88.23]
solov2_R101_3x [15.47, 29.71, 89.07]
yolact-R50-FPN [29.92, 45.36, 94.12]
yolact-R101-FPN [36.66, 48.22, 96.38]
yolact-plus-R50-FPN [40.34, 52.53, 100.39]
yolact-plus-R101-FPN [44.42, 57.12, 98.2]

APs
mask_rcnn_R_50_DC5_3x [3.97, 18.59, 47.64]
mask_rcnn_R_50_FPN_3x [5.9, 15.14, 54.75]
mask_rcnn_R_101_DC5_3x [6.12, 12.82, 49.3]
mask_rcnn_R_101_FPN_3x [6.67, 14.63, 49.79]
mask_rcnn_R_50_FPN_400ep_LSJ [5.96, 17.07, 52.73]
mask_rcnn_R_50_FPN_3x_dconv_c3-c5 [6.36, 17.8, 56.87]

In [16]:
print(SUMMARY["mask_rcnn_R_50_DC5_3x"].keys())
print('SAP: ', SUMMARY["mask_rcnn_R_50_DC5_3x"]['sap_coco'])
print()
print('BLUR:', SUMMARY["mask_rcnn_R_50_DC5_3x"]['coco_blur'])
print()
print('STYL:', SUMMARY["mask_rcnn_R_50_DC5_3x"]['stylized_coco']['1']['pixel_space'])

dict_keys(['coco_blur', 'sap_coco', 'sap_objects', 'sap_background', 'stylized_coco', 'stylized_coco_objects', 'stylized_coco_background'])
SAP:  {'segm': {'AP': [0.3585283980270518, 0.3585283511950775, 0.045228676907134106], 'AP50': [0.5743684821443005, 0.5743684368113267, 0.08058397742686073], 'AP75': [0.37970511336749, 0.3797051169270776, 0.04430688571735537], 'APs': [0.15869249244741723, 0.15869249244741723, 0.006345253395490901], 'APm': [0.39561421415206066, 0.39561421415206066, 0.048602825678017436], 'APl': [0.5464621553002346, 0.5464621553002346, 0.09381475948013723], 'AR1': [0.3052263943379814, 0.3052263943379814, 0.07037343779258629], 'AR10': [0.4612347062550921, 0.4612347062550921, 0.09151342153288901], 'AR100': [0.4801858713078062, 0.4801858713078062, 0.0922132575954935], 'ARs': [0.27628677226634757, 0.27628677226634757, 0.01353081131466407], 'ARm': [0.5285553128806741, 0.5285553128806741, 0.09622600833545418], 'ARl': [0.656166222560796, 0.656166222560796, 0.1654638682599783

## Metrics for all models on val2017

In [8]:
Cat20  = Category20[20]
Cat20c = Category20c[20]
Cat20b = Category20b[20]

colors = []
MASK_1 = Cat20c[2] # FPN,C4,DC5,DCN
MASK_2 = Cat20c[0] # SWIN
MASK_3 = Cat20c[1] # LSJ
MASK_4 = Cat20c[3] # SSL

CASK_1 = Cat20c[6] # FPN, DCN
CASK_2 = Cat20c[4] # SWIN

YOLO_1   = Cat20[8]
YOLACT_1 = Cat20[12] # Yolact
YOLACT_2 = Cat20[13] # Yolact++

DETR_1 = Cat20c[8]
SOTR_1 = Cat20[17]
SOLO_1 = Cat20[18]

BCNET_1 = Cat20b[0]

In [10]:
MASK_RCNN = [
    # FPN
    ["mask_rcnn_R_50_FPN_1x", ("Mask R-CNN", "FPN", "R50 (12)"),   MASK_1],  
    ["mask_rcnn_R_50_FPN_3x", ("Mask R-CNN", "FPN", "R50 (36)"),   MASK_1],  
    ["R_50_FPN_Supervised_6x",("Mask R-CNN", "FPN", "R50 (72)"),   MASK_1], # from ssl models (IN pretrained)
    ["R_50_FPN_Random_6x",    ("Mask R-CNN", "FPN", "r-R50 (72)"), MASK_1], # from ssl models (random init)
    ["mask_rcnn_R_101_FPN_3x",       ("Mask R-CNN", "FPN", "R101 (36)"), MASK_1],
    ["mask_rcnn_X_101_32x8d_FPN_3x", ("Mask R-CNN", "FPN", "X101 (36)"), MASK_1],
    # SWIN
    ["mask_rcnn_swin-T_1x", ("Mask R-CNN", "FPN", "Swin-T (12)"), MASK_2],
    ["mask_rcnn_swin-T_3x", ("Mask R-CNN", "FPN", "Swin-T (36)"), MASK_2],
    ["mask_rcnn_swin-S_3x", ("Mask R-CNN", "FPN", "Swin-S (36)"), MASK_2],
    # C4,DC5,DCN
    ["mask_rcnn_R_50_C4_1x",  ("Mask R-CNN", "C4", "R50 (12)"),  MASK_1 ],  
    ["mask_rcnn_R_50_C4_3x",  ("Mask R-CNN", "C4", "R50 (36)"),  MASK_1 ], 
    ["mask_rcnn_R_101_C4_3x", ("Mask R-CNN", "C4", "R101 (36)"), MASK_1 ],
    ["mask_rcnn_R_50_DC5_1x", ("Mask R-CNN", "DC5", "R50 (12)"), MASK_1 ],  
    ["mask_rcnn_R_50_DC5_3x", ("Mask R-CNN", "DC5", "R50 (36)"), MASK_1 ], 
    ["mask_rcnn_R_101_DC5_3x",("Mask R-CNN", "DC5", "R101 (36)"), MASK_1],
    ["mask_rcnn_R_50_FPN_1x_dconv_c3-c5", ("Mask R-CNN", "DCN", "R50 (12)"), MASK_1],  
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("Mask R-CNN", "DCN", "R50 (36)"), MASK_1],
    # LSJ
    ["mask_rcnn_R_50_FPN_100ep_LSJ",  ("Mask R-CNN", "LSJ", "R50 (100)"),  MASK_3],
    ["mask_rcnn_R_50_FPN_200ep_LSJ",  ("Mask R-CNN", "LSJ", "R50 (200)"),  MASK_3],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",  ("Mask R-CNN", "LSJ", "R50 (400)"),  MASK_3],
    ["mask_rcnn_R_101_FPN_100ep_LSJ", ("Mask R-CNN", "LSJ", "R101 (100)"), MASK_3],
    ["mask_rcnn_R_101_FPN_200ep_LSJ", ("Mask R-CNN", "LSJ", "R101 (200)"), MASK_3],
    ["mask_rcnn_R_101_FPN_400ep_LSJ", ("Mask R-CNN", "LSJ", "R101 (400)"), MASK_3],
    ["mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetX (100)"), MASK_3],
    ["mask_rcnn_regnetx_4gf_dds_FPN_200ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetX (200)"), MASK_3],
    ["mask_rcnn_regnetx_4gf_dds_FPN_400ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetX (400)"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetY (100)"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_200ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetY (200)"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_400ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetY (400)"), MASK_3],
    # SSL
    ["R_50_FPN_InstDis_1x", ("Mask R-CNN", "SSL", "InstDis (12)"), MASK_4],
    ["R_50_FPN_InstDis_2x", ("Mask R-CNN", "SSL", "InstDis (24)"), MASK_4],
    ["R_50_FPN_PIRL_1x",    ("Mask R-CNN", "SSL", "PIRL (12)")   , MASK_4],    
    ["R_50_FPN_PIRL_2x",    ("Mask R-CNN", "SSL", "PIRL (24)")   , MASK_4],
    ["R_50_FPN_MoCo_v1_1x", ("Mask R-CNN", "SSL", "MoCo v1 (12)"), MASK_4], 
    ["R_50_FPN_MoCo_v1_2x", ("Mask R-CNN", "SSL", "MoCo v1 (24)"), MASK_4], 
    ["R_50_FPN_MoCo_v2_2x", ("Mask R-CNN", "SSL", "MoCo v2 (24)"), MASK_4],
    ["R_50_FPN_InfoMin_1x", ("Mask R-CNN", "SSL", "InfoMin (12)"), MASK_4], 
    ["R_50_FPN_InfoMin_2x", ("Mask R-CNN", "SSL", "InfoMin (24)"), MASK_4], 
    ["R_50_FPN_InfoMin_6x", ("Mask R-CNN", "SSL", "InfoMin (72)"), MASK_4]
]

CASCADE = [
    ["cascade_mask_rcnn_R_50_FPN_1x", ("Cascade", "FPN", "R50 (12)"), CASK_1],   
    ["cascade_mask_rcnn_R_50_FPN_3x", ("Cascade", "FPN", "R50 (36)"), CASK_1],
    ["cascade_mask_rcnn_swin-T_1x",   ("Cascade", "FPN", "Swin-T (12)"), CASK_2],
    ["cascade_mask_rcnn_swin-T_3x",   ("Cascade", "FPN", "Swin-T (36)"), CASK_2],
    ["cascade_mask_rcnn_swin-S_3x",   ("Cascade", "FPN", "Swin-S (36)"), CASK_2],
    ["cascade_mask_rcnn_swin-B_3x",   ("Cascade", "FPN", "Swin-B (36)"), CASK_2],
    ["panoptic_fpn_R_101_dconv_cascade_gn_3x",          ("Cascade", "DCN", "R101 (36)"), CASK_1], # panoptic  
    ["cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv", ("Cascade", "DCN", "X151 (53)"), CASK_1]
]

BMask = [  
    ["bmask_rcnn_r50_1x",  ("BMask", "FPN", "R50 (12)" ), MASK_1],
    ["bmask_rcnn_r101_1x", ("BMask", "FPN", "R101 (12)"), MASK_1],
    ["cascade_bmask_rcnn_r50_1x",     ("BMask", "FPN", "Casc. R50 (12)" ), CASK_1],
    ["cascade_bmask_rcnn_r101_1x",    ("BMask", "FPN", "Casc. R101 (12)"), CASK_1]
]
BCNet = [
    #["fcos_imprv_R_101_FPN", ("BCNet", "FPN", "FCOS R101 (24)"), BCNET_1]
    ["fcos_imprv_R_101_FPN", ("BCN", "FPN", "FCOS R101 (24)"), BCNET_1] # For Distance Matrices
]

YOLO = [
    ["yolov3",     ("YOLO", "FPN",  "D52 (267)")    , YOLO_1], 
    ["yolov4",     ("YOLO", "PAN",  "CD53 (267)")   , YOLO_1],
    ["yolov4-csp", ("YOLO", "CPAN", "CD53 (267)")   , YOLO_1],
    ["yolov4-p5",  ("YOLO", "CPAN", "CD53-p5 (267)"), YOLO_1],
    ["yolov4-p6",  ("YOLO", "CPAN", "CD53-p6 (267)"), YOLO_1],
]
YOLACT =[
    ["yolact-R50-FPN",       ("YOLACT", "FPN", "R50 (53)")   ,YOLACT_1],
    ["yolact-R101-FPN",      ("YOLACT", "FPN", "R101 (53)")  ,YOLACT_1],
    ["yolact-plus-R50-FPN",  ("YOLACT", "DCN", "++R50 (53)") ,YOLACT_2],
    ["yolact-plus-R101-FPN", ("YOLACT", "DCN", "++R101 (53)"),YOLACT_2],
]
DETR=[
    ["detr_r50",     ("DETR", "FPN", "R50 (325)") , DETR_1], 
    ["detr_r101",    ("DETR", "FPN", "R101 (352)") , DETR_1],
    ["detr_r50_dc5", ("DETR", "DC5", "R50 (325)"), DETR_1]
]
SOTR=[
    ["sotr_R101",     ("SOTR", "FPN", "R101 (20)"), SOTR_1], 
    ["sotr_R101_dcn", ("SOTR", "DCN", "R101 (20)"), SOTR_1]
]
SOLO=[
    ["solov2_R50_3x",  ("SOLO", "FPN", "R50 (36)"),  SOLO_1],
    ["solov2_R101_3x", ("SOLO", "FPN", "R101 (36)"), SOLO_1]
]

MODELS_FACTORS_COLORS = []
MODELS_FACTORS_COLORS.extend(MASK_RCNN)
MODELS_FACTORS_COLORS.extend(BMask)
MODELS_FACTORS_COLORS.extend(CASCADE)
MODELS_FACTORS_COLORS.extend(YOLO)
MODELS_FACTORS_COLORS.extend(YOLACT)
MODELS_FACTORS_COLORS.extend(DETR)
MODELS_FACTORS_COLORS.extend(BCNet)
MODELS_FACTORS_COLORS.extend(SOTR)
MODELS_FACTORS_COLORS.extend(SOLO)
print(len(MODELS_FACTORS_COLORS))

68


In [11]:
from bokeh.models import BoxAnnotation

In [12]:


def compare_val2017(models: list, factors: list, colors: list, stat:str = "AP", iou_type: str = 'segm', dataset=STYLIZED_COCO):
        
    # prepare figure
    p = figure(x_range = FactorRange(*factors),
               y_axis_label=str(stat)+ " (bbox)",
               plot_height=250, plot_width=1150, #990,
               toolbar_location=None, tools="",output_backend='svg')

    # prepare data
    ys = []
    for m in models:
        
        if iou_type == 'segm' and 'yolo' in m: 
            iou = 'bbox'
        else:
            iou = iou_type
        y = SUMMARY[m][dataset]['1']['pixel_space'][iou][stat][0]
        ys.append(y)
        
    # plot data
    p.vbar(x=factors, top=ys, width=0.65, alpha=1.0, line_alpha=1.0, color=colors)
    
    # plot value info
    text_font_style = ['normal' for f in factors]
    if iou_type == 'segm':
        text_font_style[8] ='bold'
        text_font_style[25]='bold'
        text_font_style[28]='bold'
        text_font_style[47]='bold'
        text_font_style[48]='bold'
        text_font_style[55]='bold'
    if iou_type == 'bbox':
        text_font_style[8] ='bold'
        text_font_style[22]='bold'
        text_font_style[25]='bold'
        text_font_style[28]='bold'
        text_font_style[47]='bold'
        text_font_style[48]='bold'
        text_font_style[55]='bold'
    
#    p.text(x=factors, y=[0.525 for y in ys], text=list(map(str, [np.round(y*100,2) for y in ys])), 
    p.text(x=factors, y=[0.7525 for y in ys], text=list(map(str, [np.round(y*100,2) for y in ys])), 
           angle=math.pi/2, text_font_size='10px', x_offset=4, y_offset=-8, text_font_style=text_font_style)
    
    # mark yolo since its BBox AP
    p.add_layout(BoxAnnotation(left=59.9, right=69.6, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))
    
    # configure plot
    p.y_range.start = 0
    p.y_range.end   = 1.1
    #p.y_range.end   = 0.799
    p.x_range.range_padding    = 0.03 # left and right of x-range                  # default = 0.1
    p.x_range.factor_padding   = -0.1 # between highest level factors (factors)    # default = 0.0
    p.x_range.subgroup_padding = 1.2  # between middle  level factors (sub-groups) # default = 0.8
    p.x_range.group_padding    = 2.4  # between lowest  level factors (groups)     # default = 1.4
    p.xgrid.grid_line_color    = None
    p.yaxis.ticker=FixedTicker(ticks=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
    
    # a little hack since svg export doesn't align rotated xlabels correctly
    # create space but hide xlabels
    p.xaxis.major_label_orientation = math.pi/2
    p.xaxis.major_label_text_alpha = 0
    # fake labels as text which align correctly when rotated
    p.text(x=factors, y=[-0.075 for y in ys], text=[f[2] for f in factors],
           text_align='right', angle=math.pi/2, text_font_size='10px', 
           x_offset=4, y_offset=0, text_font_style=text_font_style)
        
    p.xaxis.major_label_standoff = -2
    p.yaxis.axis_label_standoff = 0
    
    # show plot
    show(p)
    
    # for saving
    return p

In [13]:
models, factors, colors = map(list,(zip(*MODELS_FACTORS_COLORS)))

for stat in ['AP', 'APs','APm','APl']:
#for stat in ['AP']:
    p    = compare_val2017(models, factors, colors, stat=stat, iou_type='segm')
    #export_svg(p, filename=f"../plots/draft_v2/a_val2017_{stat}_bbox.svg")

## Distance Matrix 

In [11]:
MASK_RCNN = [
    # FPN
    ["mask_rcnn_R_50_FPN_1x", ("Mask R-CNN", "FPN", "R50 (12)"),   MASK_1],  
    ["mask_rcnn_R_50_FPN_3x", ("Mask R-CNN", "FPN", "R50 (36)"),   MASK_1],  
    ["R_50_FPN_Supervised_6x",("Mask R-CNN", "FPN", "R50 (72)"),   MASK_1], # from ssl models (IN pretrained)
    ["R_50_FPN_Random_6x",    ("Mask R-CNN", "FPN", "r-R50 (72)"), MASK_1], # from ssl models (random init)
    ["mask_rcnn_R_101_FPN_3x",       ("Mask R-CNN", "FPN", "R101 (36)"), MASK_1],
    ["mask_rcnn_X_101_32x8d_FPN_3x", ("Mask R-CNN", "FPN", "X101 (36)"), MASK_1],
    # C4,DC5,DCN
    ["mask_rcnn_R_50_C4_1x",  ("Mask R-CNN", "C4", "R50 (12)"),  MASK_1 ],  
    ["mask_rcnn_R_50_C4_3x",  ("Mask R-CNN", "C4", "R50 (36)"),  MASK_1 ], 
    ["mask_rcnn_R_101_C4_3x", ("Mask R-CNN", "C4", "R101 (36)"), MASK_1 ],
    ["mask_rcnn_R_50_DC5_1x", ("Mask R-CNN", "DC5", "R50 (12)"), MASK_1 ],  
    ["mask_rcnn_R_50_DC5_3x", ("Mask R-CNN", "DC5", "R50 (36)"), MASK_1 ], 
    ["mask_rcnn_R_101_DC5_3x",("Mask R-CNN", "DC5", "R101 (36)"), MASK_1],
    ["mask_rcnn_R_50_FPN_1x_dconv_c3-c5", ("Mask R-CNN", "DCN", "R50 (12)"), MASK_1],  
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("Mask R-CNN", "DCN", "R50 (36)"), MASK_1],
    # LSJ
    ["mask_rcnn_R_50_FPN_100ep_LSJ",  ("Mask R-CNN", "LSJ", "R50 (100)"),  MASK_3],
    ["mask_rcnn_R_50_FPN_200ep_LSJ",  ("Mask R-CNN", "LSJ", "R50 (200)"),  MASK_3],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",  ("Mask R-CNN", "LSJ", "R50 (400)"),  MASK_3],
    ["mask_rcnn_R_101_FPN_100ep_LSJ", ("Mask R-CNN", "LSJ", "R101 (100)"), MASK_3],
    ["mask_rcnn_R_101_FPN_200ep_LSJ", ("Mask R-CNN", "LSJ", "R101 (200)"), MASK_3],
    ["mask_rcnn_R_101_FPN_400ep_LSJ", ("Mask R-CNN", "LSJ", "R101 (400)"), MASK_3],
    ["mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetX (100)"), MASK_3],
    ["mask_rcnn_regnetx_4gf_dds_FPN_200ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetX (200)"), MASK_3],
    ["mask_rcnn_regnetx_4gf_dds_FPN_400ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetX (400)"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetY (100)"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_200ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetY (200)"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_400ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetY (400)"), MASK_3],
    # SSL
    ["R_50_FPN_InstDis_1x", ("Mask R-CNN", "SSL", "InstDis (12)"), MASK_4],
    ["R_50_FPN_InstDis_2x", ("Mask R-CNN", "SSL", "InstDis (24)"), MASK_4],
    ["R_50_FPN_PIRL_1x",    ("Mask R-CNN", "SSL", "PIRL (12)")   , MASK_4],    
    ["R_50_FPN_PIRL_2x",    ("Mask R-CNN", "SSL", "PIRL (24)")   , MASK_4],
    ["R_50_FPN_MoCo_v1_1x", ("Mask R-CNN", "SSL", "MoCo v1 (12)"), MASK_4], 
    ["R_50_FPN_MoCo_v1_2x", ("Mask R-CNN", "SSL", "MoCo v1 (24)"), MASK_4], 
    ["R_50_FPN_MoCo_v2_2x", ("Mask R-CNN", "SSL", "MoCo v2 (24)"), MASK_4],
    ["R_50_FPN_InfoMin_1x", ("Mask R-CNN", "SSL", "InfoMin (12)"), MASK_4], 
    ["R_50_FPN_InfoMin_2x", ("Mask R-CNN", "SSL", "InfoMin (24)"), MASK_4], 
    ["R_50_FPN_InfoMin_6x", ("Mask R-CNN", "SSL", "InfoMin (72)"), MASK_4]
]
SWIN = [
     # SWIN
    ["mask_rcnn_swin-T_1x", ("Mask R-CNN", "FPN", "Swin-T (12)"), MASK_2],
    ["mask_rcnn_swin-T_3x", ("Mask R-CNN", "FPN", "Swin-T (36)"), MASK_2],
    ["mask_rcnn_swin-S_3x", ("Mask R-CNN", "FPN", "Swin-S (36)"), MASK_2],
    ["cascade_mask_rcnn_swin-T_1x",   ("Cascade", "FPN", "Swin-T (12)"), CASK_2],
    ["cascade_mask_rcnn_swin-T_3x",   ("Cascade", "FPN", "Swin-T (36)"), CASK_2],
    ["cascade_mask_rcnn_swin-S_3x",   ("Cascade", "FPN", "Swin-S (36)"), CASK_2],
    ["cascade_mask_rcnn_swin-B_3x",   ("Cascade", "FPN", "Swin-B (36)"), CASK_2]   
]
CASCADE = [
    ["cascade_mask_rcnn_R_50_FPN_1x", ("Cascade", "FPN", "R50 (12)"), CASK_1],   
    ["cascade_mask_rcnn_R_50_FPN_3x", ("Cascade", "FPN", "R50 (36)"), CASK_1],
    ["panoptic_fpn_R_101_dconv_cascade_gn_3x",          ("Cascade", "DCN", "R101 (36)"), CASK_1], # panoptic  
    ["cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv", ("Cascade", "DCN", "X151 (53)"), CASK_1]
]

BMask = [  
    ["bmask_rcnn_r50_1x",  ("BMask", "FPN", "R50 (12)" ), MASK_1],
    ["bmask_rcnn_r101_1x", ("BMask", "FPN", "R101 (12)"), MASK_1],
    ["cascade_bmask_rcnn_r50_1x",     ("BMask", "FPN", "Casc. R50 (12)" ), CASK_1],
    ["cascade_bmask_rcnn_r101_1x",    ("BMask", "FPN", "Casc. R101 (12)"), CASK_1]
]
BCNet = [
    #["fcos_imprv_R_101_FPN", ("BCNet", "FPN", "FCOS R101 (24)"), BCNET_1]
    ["fcos_imprv_R_101_FPN", ("BCN", "FPN", "FCOS R101 (24)"), BCNET_1] # For Distance Matrices
]

YOLO = [
    ["yolov3",     ("YOLO", "FPN",  "D52 (267)")    , YOLO_1], 
    ["yolov4",     ("YOLO", "PAN",  "CD53 (267)")   , YOLO_1],
    ["yolov4-csp", ("YOLO", "CPAN", "CD53 (267)")   , YOLO_1],
    ["yolov4-p5",  ("YOLO", "CPAN", "CD53-p5 (267)"), YOLO_1],
    ["yolov4-p6",  ("YOLO", "CPAN", "CD53-p6 (267)"), YOLO_1],
]
YOLACT =[
    ["yolact-R50-FPN",       ("YOLACT", "FPN", "R50 (53)")   ,YOLACT_1],
    ["yolact-R101-FPN",      ("YOLACT", "FPN", "R101 (53)")  ,YOLACT_1]
]
YOLACT_2 = [
    ["yolact-plus-R50-FPN",  ("YOLACT", "DCN", "++R50 (53)") ,YOLACT_2],
    ["yolact-plus-R101-FPN", ("YOLACT", "DCN", "++R101 (53)"),YOLACT_2]
]
DETR=[
    ["detr_r50",     ("DETR", "FPN", "R50 (325)") , DETR_1], 
    ["detr_r101",    ("DETR", "FPN", "R101 (352)") , DETR_1],
    ["detr_r50_dc5", ("DETR", "DC5", "R50 (325)"), DETR_1]
]
SOTR=[
    ["sotr_R101",     ("SOTR", "FPN", "R101 (20)"), SOTR_1], 
    ["sotr_R101_dcn", ("SOTR", "DCN", "R101 (20)"), SOTR_1]
]
SOLO=[
    ["solov2_R50_3x",  ("SOLO", "FPN", "R50 (36)"),  SOLO_1],
    ["solov2_R101_3x", ("SOLO", "FPN", "R101 (36)"), SOLO_1]
]


In [16]:
stats=['AP','APs','APm','APl']

models = []
models.extend(MASK_RCNN)
models.extend(SWIN)
models.extend(BMask)
models.extend(CASCADE)
models.extend(YOLACT)
models.extend(YOLACT_2)
models.extend(DETR)
models.extend(BCNet)
models.extend(SOTR)
models.extend(SOLO)

models, factors, colors = map(list,(zip(*models)))      
dist_mat, min_d, max_d  = calc_mean_distance_matrix(models, STYLIZED_COCO_OBJECTS, stats, 'segm')

d = dist_mat.reshape((63,63))

start = 0
f = []
o = []
for l in [len(x) for x in [MASK_RCNN,SWIN,BMask,CASCADE,YOLACT,YOLACT_2,DETR,BCNet,SOTR,SOLO]]:
    # within the framework
    f_slice = d[start:start+l,start:start+l]
    f_slice = f_slice[f_slice != 0] if len(f_slice) > 1 else f_slice
    f.append(f_slice.flatten())

    # outside the framework
    o_1 = d[start:start+l,   :start]
    o_2 = d[start:start+l, start+l:]
    o_slice = np.concatenate((o_1,o_2),axis=1)
    o_slice = o_slice[o_slice != 0]
    o.append(o_slice.flatten())

    start += l

x = []
y = []
for i,j in zip(f,o):
    print(f"{np.mean(i):0.2f} : {np.mean(j):0.2f}")
    x.append(np.mean(i))
    y.append(np.mean(j))
    #x.extend(i)
    #y.extend(j)

print(f"\n{np.mean(x):0.2f}+-{np.std(x):0.2f}")
print(f"\n{np.mean(y):0.2f}+-{np.std(y):0.2f}")

0.13 : 0.23
0.08 : 0.19
0.06 : 0.16
0.18 : 0.18
0.12 : 0.34
0.05 : 0.52
0.09 : 0.21
0.00 : 0.16
0.13 : 0.34
0.05 : 0.27

0.09+-0.05

0.26+-0.11


In [50]:
np.concatenate(([1],[2]))

array([1, 2])

In [13]:
stats=['AP','APs','APm','APl']

means = []
stds  = []
for m in [MASK_RCNN, SWIN, BMask, CASCADE, YOLACT, YOLACT_2, DETR, BCNet, SOTR, SOLO]:
#for m in [YOLACT, YOLACT_2]:    

    models, factors, colors = map(list,(zip(*m)))      
    dist_mat, min_d, max_d = calc_mean_distance_matrix(models, STYLIZED_COCO, stats, 'segm')
    
    m = dist_mat
    m = m[m != 0] if len(m) > 1 else m
    means.append(np.mean(m))
    stds.append(np.std(m))
    print(f"{np.mean(m)}+-{np.std(m)}")

print(f"Average distance within frameworks: {np.mean(means)}+-{np.mean(stds)}")

#w,h, = len(MODELS_FACTORS_COLORS),len(MODELS_FACTORS_COLORS)
#dist_mat = dist_mat.reshape((w,h))

0.10963265627066117+-0.062483177376967326
0.08347073714076099+-0.04061016878092975
0.040202008766671626+-0.014949972617552858
0.19414816596337858+-0.11115382965726518
0.1411698084116907+-0.0
0.04846132969605249+-0.0
0.07596422310296339+-0.02515666225725074
0.0+-0.0
0.15775861580251366+-0.0
0.02899609405823421+-0.0
Average distance within frameworks: 0.08798036392129269+-0.025435381068996586


In [16]:
models = ["a","b"]#.extend(['v'])
models

['a', 'b']

In [109]:

m1 = []

MODELS_FACTORS_COLORS = []
MODELS_FACTORS_COLORS.extend(MASK_RCNN)
MODELS_FACTORS_COLORS.extend(SWIN)
MODELS_FACTORS_COLORS.extend(BMask)
MODELS_FACTORS_COLORS.extend(CASCADE)
MODELS_FACTORS_COLORS.extend(YOLACT)
MODELS_FACTORS_COLORS.extend(DETR)
MODELS_FACTORS_COLORS.extend(BCNet)
MODELS_FACTORS_COLORS.extend(SOTR)
MODELS_FACTORS_COLORS.extend(SOLO)

stats=['AP','APs','APm','APl']

models, factors, colors = map(list,(zip(*MODELS_FACTORS_COLORS)))      
dist_mat, min_d, max_d = calc_mean_distance_matrix(models, STYLIZED_COCO, stats, 'segm')

d = dist_mat.reshape((len(MODELS_FACTORS_COLORS),len(MODELS_FACTORS_COLORS)))

means = []
stds  = []
def sliced_mean_std(start,stop,d):
    d = d[start:stop, start:stop]
    d = d[d != 0]
    m,s = np.mean(d), np.std(d)
    print(f"{m}+-{s}")
    return m,s

# Mask RCNN
m,s = sliced_mean_std(0,len(MASK_RCNN), d)
means.append(m)
stds.append(s)

# SWIN
m,s = sliced_mean_std(len(MASK_RCNN), len(MASK_RCNN) + len(SWIN), d)
means.append(m)
stds.append(s)

# SWIN
m,s = sliced_mean_std(len(MASK_RCNN), len(MASK_RCNN) + len(SWIN), d)
means.append(m)
stds.append(s)

m,s = sliced_mean_std(len(MASK_RCNN), None, d)

0.10963265627066117+-0.062483177376967326
0.08347073714076099+-0.04061016878092975
0.08347073714076099+-0.04061016878092975
0.22037937890020964+-0.11958836377192715


In [10]:
MASK_RCNN = [
    # FPN
    ["mask_rcnn_R_50_FPN_1x", ("Mask R-CNN", "FPN", "R50 (12)"),   MASK_1],  
    ["mask_rcnn_R_50_FPN_3x", ("Mask R-CNN", "FPN", "R50 (36)"),   MASK_1],  
    ["R_50_FPN_Supervised_6x",("Mask R-CNN", "FPN", "R50 (72)"),   MASK_1], # from ssl models (IN pretrained)
    ["R_50_FPN_Random_6x",    ("Mask R-CNN", "FPN", "r-R50 (72)"), MASK_1], # from ssl models (random init)
    ["mask_rcnn_R_101_FPN_3x",       ("Mask R-CNN", "FPN", "R101 (36)"), MASK_1],
    ["mask_rcnn_X_101_32x8d_FPN_3x", ("Mask R-CNN", "FPN", "X101 (36)"), MASK_1],
    # SWIN
    ["mask_rcnn_swin-T_1x", ("Mask R-CNN", "FPN", "Swin-T (12)"), MASK_2],
    ["mask_rcnn_swin-T_3x", ("Mask R-CNN", "FPN", "Swin-T (36)"), MASK_2],
    ["mask_rcnn_swin-S_3x", ("Mask R-CNN", "FPN", "Swin-S (36)"), MASK_2],
    # C4,DC5,DCN
    ["mask_rcnn_R_50_C4_1x",  ("Mask R-CNN", "C4", "R50 (12)"),  MASK_1 ],  
    ["mask_rcnn_R_50_C4_3x",  ("Mask R-CNN", "C4", "R50 (36)"),  MASK_1 ], 
    ["mask_rcnn_R_101_C4_3x", ("Mask R-CNN", "C4", "R101 (36)"), MASK_1 ],
    ["mask_rcnn_R_50_DC5_1x", ("Mask R-CNN", "DC5", "R50 (12)"), MASK_1 ],  
    ["mask_rcnn_R_50_DC5_3x", ("Mask R-CNN", "DC5", "R50 (36)"), MASK_1 ], 
    ["mask_rcnn_R_101_DC5_3x",("Mask R-CNN", "DC5", "R101 (36)"), MASK_1],
    ["mask_rcnn_R_50_FPN_1x_dconv_c3-c5", ("Mask R-CNN", "DCN", "R50 (12)"), MASK_1],  
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("Mask R-CNN", "DCN", "R50 (36)"), MASK_1],
    # LSJ
    ["mask_rcnn_R_50_FPN_100ep_LSJ",  ("Mask R-CNN", "LSJ", "R50 (100)"),  MASK_3],
    ["mask_rcnn_R_50_FPN_200ep_LSJ",  ("Mask R-CNN", "LSJ", "R50 (200)"),  MASK_3],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",  ("Mask R-CNN", "LSJ", "R50 (400)"),  MASK_3],
    ["mask_rcnn_R_101_FPN_100ep_LSJ", ("Mask R-CNN", "LSJ", "R101 (100)"), MASK_3],
    ["mask_rcnn_R_101_FPN_200ep_LSJ", ("Mask R-CNN", "LSJ", "R101 (200)"), MASK_3],
    ["mask_rcnn_R_101_FPN_400ep_LSJ", ("Mask R-CNN", "LSJ", "R101 (400)"), MASK_3],
    ["mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetX (100)"), MASK_3],
    ["mask_rcnn_regnetx_4gf_dds_FPN_200ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetX (200)"), MASK_3],
    ["mask_rcnn_regnetx_4gf_dds_FPN_400ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetX (400)"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetY (100)"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_200ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetY (200)"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_400ep_LSJ", ("Mask R-CNN", "LSJ", "RegNetY (400)"), MASK_3],
    # SSL
    ["R_50_FPN_InstDis_1x", ("Mask R-CNN", "SSL", "InstDis (12)"), MASK_4],
    ["R_50_FPN_InstDis_2x", ("Mask R-CNN", "SSL", "InstDis (24)"), MASK_4],
    ["R_50_FPN_PIRL_1x",    ("Mask R-CNN", "SSL", "PIRL (12)")   , MASK_4],    
    ["R_50_FPN_PIRL_2x",    ("Mask R-CNN", "SSL", "PIRL (24)")   , MASK_4],
    ["R_50_FPN_MoCo_v1_1x", ("Mask R-CNN", "SSL", "MoCo v1 (12)"), MASK_4], 
    ["R_50_FPN_MoCo_v1_2x", ("Mask R-CNN", "SSL", "MoCo v1 (24)"), MASK_4], 
    ["R_50_FPN_MoCo_v2_2x", ("Mask R-CNN", "SSL", "MoCo v2 (24)"), MASK_4],
    ["R_50_FPN_InfoMin_1x", ("Mask R-CNN", "SSL", "InfoMin (12)"), MASK_4], 
    ["R_50_FPN_InfoMin_2x", ("Mask R-CNN", "SSL", "InfoMin (24)"), MASK_4], 
    ["R_50_FPN_InfoMin_6x", ("Mask R-CNN", "SSL", "InfoMin (72)"), MASK_4]
]

CASCADE = [
    ["cascade_mask_rcnn_R_50_FPN_1x", ("Cascade", "FPN", "R50 (12)"), CASK_1],   
    ["cascade_mask_rcnn_R_50_FPN_3x", ("Cascade", "FPN", "R50 (36)"), CASK_1],
    ["cascade_mask_rcnn_swin-T_1x",   ("Cascade", "FPN", "Swin-T (12)"), CASK_2],
    ["cascade_mask_rcnn_swin-T_3x",   ("Cascade", "FPN", "Swin-T (36)"), CASK_2],
    ["cascade_mask_rcnn_swin-S_3x",   ("Cascade", "FPN", "Swin-S (36)"), CASK_2],
    ["cascade_mask_rcnn_swin-B_3x",   ("Cascade", "FPN", "Swin-B (36)"), CASK_2],
    ["panoptic_fpn_R_101_dconv_cascade_gn_3x",          ("Cascade", "DCN", "R101 (36)"), CASK_1], # panoptic  
    ["cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv", ("Cascade", "DCN", "X151 (53)"), CASK_1]
]

BMask = [  
    ["bmask_rcnn_r50_1x",  ("BMask", "FPN", "R50 (12)" ), MASK_1],
    ["bmask_rcnn_r101_1x", ("BMask", "FPN", "R101 (12)"), MASK_1],
    ["cascade_bmask_rcnn_r50_1x",     ("BMask", "FPN", "Casc. R50 (12)" ), CASK_1],
    ["cascade_bmask_rcnn_r101_1x",    ("BMask", "FPN", "Casc. R101 (12)"), CASK_1]
]
BCNet = [
    #["fcos_imprv_R_101_FPN", ("BCNet", "FPN", "FCOS R101 (24)"), BCNET_1]
    ["fcos_imprv_R_101_FPN", ("BCN", "FPN", "FCOS R101 (24)"), BCNET_1] # For Distance Matrices
]

YOLO = [
    ["yolov3",     ("YOLO", "FPN",  "D52 (267)")    , YOLO_1], 
    ["yolov4",     ("YOLO", "PAN",  "CD53 (267)")   , YOLO_1],
    ["yolov4-csp", ("YOLO", "CPAN", "CD53 (267)")   , YOLO_1],
    ["yolov4-p5",  ("YOLO", "CPAN", "CD53-p5 (267)"), YOLO_1],
    ["yolov4-p6",  ("YOLO", "CPAN", "CD53-p6 (267)"), YOLO_1],
]
YOLACT =[
    ["yolact-R50-FPN",       ("YOLACT", "FPN", "R50 (53)")   ,YOLACT_1],
    ["yolact-R101-FPN",      ("YOLACT", "FPN", "R101 (53)")  ,YOLACT_1],
    ["yolact-plus-R50-FPN",  ("YOLACT", "DCN", "++R50 (53)") ,YOLACT_2],
    ["yolact-plus-R101-FPN", ("YOLACT", "DCN", "++R101 (53)"),YOLACT_2],
]
DETR=[
    ["detr_r50",     ("DETR", "FPN", "R50 (325)") , DETR_1], 
    ["detr_r101",    ("DETR", "FPN", "R101 (352)") , DETR_1],
    ["detr_r50_dc5", ("DETR", "DC5", "R50 (325)"), DETR_1]
]
SOTR=[
    ["sotr_R101",     ("SOTR", "FPN", "R101 (20)"), SOTR_1], 
    ["sotr_R101_dcn", ("SOTR", "DCN", "R101 (20)"), SOTR_1]
]
SOLO=[
    ["solov2_R50_3x",  ("SOLO", "FPN", "R50 (36)"),  SOLO_1],
    ["solov2_R101_3x", ("SOLO", "FPN", "R101 (36)"), SOLO_1]
]

MODELS_FACTORS_COLORS = []
MODELS_FACTORS_COLORS.extend(MASK_RCNN)
MODELS_FACTORS_COLORS.extend(BMask)
MODELS_FACTORS_COLORS.extend(CASCADE)
MODELS_FACTORS_COLORS.extend(YOLO)
MODELS_FACTORS_COLORS.extend(YOLACT)
MODELS_FACTORS_COLORS.extend(DETR)
MODELS_FACTORS_COLORS.extend(BCNet)
MODELS_FACTORS_COLORS.extend(SOTR)
MODELS_FACTORS_COLORS.extend(SOLO)
print(len(MODELS_FACTORS_COLORS))

68


In [9]:
def calc_mean_distance_matrix(models, dataset, stats, iou_type):
    
    distance_matrices = []
    for stat in stats:

        # calculate relative performance per model
        model_scores = []
        for m in models:
            if 'yolo' in m: continue
            
            abs_y = SUMMARY[m][dataset]['1']['feature_space'][iou_type][stat] #[alpha]

            # the uncorrupted coco dataset is alpha=0 in pixel space
            coco_score = SUMMARY[m][dataset]['1']['pixel_space'][iou_type][stat][0]
            rel_y = abs_y / coco_score
            model_scores.append(rel_y)

        # euclidian distance between models
        euclidian_distances = []
        for s1 in model_scores:
            for s2 in model_scores:
                euclidian_distances.append(np.sqrt(np.sum((s1 - s2)**2)))

        distance_matrices.append(euclidian_distances)
        
    # element wise mean over stats, min and max    
    mean_distance_matrix = np.mean(distance_matrices, axis=0)
    return mean_distance_matrix, np.min(mean_distance_matrix), np.max(mean_distance_matrix)

def plot_mean_distance_matrix(models, factors, DATASETS, stats=['AP','APs','APm','APl'], iou_type='segm', PRINT_VALUES=False):
    
    # calculate the avg distance matrix for all models
    
    matrices,mins,maxs = [], [], []
    
    for dataset in DATASETS:
        dist_mat, min_d, max_d = calc_mean_distance_matrix(models, dataset, stats, iou_type)
        matrices.append(dist_mat)
        mins.append(min_d)
        maxs.append(max_d)
        
    min_d = min(mins)
    max_d = max(maxs)

    # prepare x,y from factors
    x,y = [], []
    for m1 in factors:
        for m2 in factors:
            x.append(m1)
            y.append(m2)
            
    # prepare color mappings and color bar
    pallete = Turbo256 #magma(256)
    mapper    = LinearColorMapper(palette=pallete, low=min_d, high=max_d)
    color_bar = ColorBar(color_mapper=mapper, location='right', width=10, height=200,
                     major_tick_line_color='black',major_tick_in=10,major_tick_out=5, 
                     title='Euclidian distance', label_standoff=10)

    plots = []
    for dist_mat in matrices:
    
        colors  = [pallete[int(np.round((d / max_d) * 255, 0))] for d in dist_mat]
        
        # prepare figure
        p = figure(x_range = FactorRange(*factors), 
                   y_range = FactorRange(*reversed(factors)),
                   plot_width = 960, plot_height = 900,
                   toolbar_location=None, tools="", output_backend='svg')

        # plot data
        p.rect(x=x, y=y, width=1, height=1, line_color=None, fill_color=colors)

        if PRINT_VALUES: # plots all numbers
            p.text(x=x, y=y, text=[str(np.round(v,2)) for v in e], color='white', text_font_size= '4px',
                   text_align='center',text_baseline='middle')
        else: # only plots diagonal
            p.text(x=factors, y=factors, text=[0 for f in factors], color='white', text_font_size= '5px',
                   text_align='center',text_baseline='middle')

        p.add_layout(color_bar, 'right')

        # mark model pairs by epochs
        if True:
            p.rect(x=[factors[1]], y=[factors[1]], width=3, height=3, line_color='yellow', fill_color=None)
            p.rect(x=[ 7.0], y=[65.3], width=2, height=2, line_color='yellow', fill_color=None)
            p.rect(x=[10.1], y=[62.2], width=2, height=2, line_color='yellow', fill_color=None)
            p.rect(x=[13.2], y=[59.1], width=2, height=2, line_color='yellow', fill_color=None)
            p.rect(x=[16.3], y=[56.0], width=2, height=2, line_color='yellow', fill_color=None)
            p.rect(x=[18.9], y=[53.4], width=3, height=3, line_color='yellow', fill_color=None)
            p.rect(x=[21.9], y=[50.3], width=3, height=3, line_color='yellow', fill_color=None)
            p.rect(x=[24.9], y=[47.3], width=3, height=3, line_color='yellow', fill_color=None)
            p.rect(x=[27.9], y=[44.4], width=3, height=3, line_color='yellow', fill_color=None)
            p.rect(x=[30.5], y=[41.8], width=2, height=2, line_color='yellow', fill_color=None)
            p.rect(x=[32.5], y=[39.7], width=2, height=2, line_color='yellow', fill_color=None)
            p.rect(x=[34.5], y=[37.7], width=2, height=2, line_color='yellow', fill_color=None)
            p.rect(x=[38.0], y=[34.3], width=3, height=3, line_color='yellow', fill_color=None)
            p.rect(x=[46.9], y=[25.4], width=2, height=2, line_color='yellow', fill_color=None)
            p.rect(x=[48.9], y=[23.4], width=2, height=2, line_color='yellow', fill_color=None)

        # style the plot and axis
        # x-axis
        p.xaxis.major_label_text_alpha     = 0
        p.xaxis.major_label_orientation    = math.pi/2
        p.xaxis.subgroup_label_orientation = math.pi/2
        p.text(x=factors, y=[0 for y in factors], text=[f[2] for f in factors], text_align='right', 
               angle=math.pi/2, text_font_size='10px', x_offset=5, y_offset=8)
        # y-axis
        p.yaxis.major_label_text_alpha     = 0
        p.yaxis.subgroup_label_orientation = 'horizontal' #@math.pi/2
        p.text(x=[0 for y in factors], y=list(reversed(factors)), text=[f[2] for f in reversed(factors)], text_align='right', 
               angle=0, text_font_size='10px', x_offset=-10, y_offset=5)

        # both axis formatting
        p.x_range.subgroup_padding = 0.1  # between middle  level factors (sub-groups) # default = 0.8
        p.y_range.subgroup_padding = 0.1
        p.x_range.group_padding    = 1.2  # between lowest  level factors (groups)     # default = 1.4
        p.y_range.group_padding    = 1.2

        show(p)
        plots.append(p)
    return plots

In [16]:
MODELS_FACTORS_COLORS = []
MODELS_FACTORS_COLORS.extend(MASK_RCNN)
MODELS_FACTORS_COLORS.extend(BMask)
MODELS_FACTORS_COLORS.extend(CASCADE)
#MODELS_FACTORS_COLORS.extend(YOLO)
MODELS_FACTORS_COLORS.extend(YOLACT)
MODELS_FACTORS_COLORS.extend(DETR)
MODELS_FACTORS_COLORS.extend(BCNet)
MODELS_FACTORS_COLORS.extend(SOTR)
MODELS_FACTORS_COLORS.extend(SOLO)

print(len(MODELS_FACTORS_COLORS))

models, factors, colors = map(list,(zip(*MODELS_FACTORS_COLORS)))

ps,po,pb = plot_mean_distance_matrix(models, factors, [STYLIZED_COCO, STYLIZED_COCO_OBJECTS, STYLIZED_COCO_BACKGROUND])
#ps = plot_mean_distance_matrix(models, factors, [STYLIZED_COCO])

#export_svg(ps, filename=f"../plots/draft_v2/dm_s-coco.svg")
#export_svg(po, filename=f"../plots/draft_v2/dm_s-obj.svg")
#export_svg(pb, filename=f"../plots/draft_v2/dm_s-bg.svg")

## Compare R50 and R101 by framework

In [8]:
R50_MODELS = [
    
    ["mask_rcnn_R_50_FPN_1x",         ("α=0", "M-RCNN"),   MASK_1],  
    #["R_50_FPN_Supervised_6x",        ("α=0", "M-RCNN"),   MASK_1],
    #["R_50_FPN_Random_6x",            ("α=0", "rM-RCNN"),    MASK_1], 
    #["mask_rcnn_R_50_FPN_100ep_LSJ",  ("α=0", "lM-RCNN"),   MASK_3],
    ["bmask_rcnn_r50_1x",             ("α=0", "BMask"),    MASK_1],  
    ["cascade_bmask_rcnn_r50_1x",     ("α=0", "C-BMask"),  CASK_1],
    ["cascade_mask_rcnn_R_50_FPN_1x", ("α=0", "Cascade"),  CASK_1],
    ["yolact-R50-FPN",                ("α=0", "YOLACT"), YOLACT_1],
    ["yolact-plus-R50-FPN",           ("α=0", "+YOLACT"),YOLACT_2],
    ["detr_r50",                      ("α=0", "DETR"),     DETR_1], 
    ["fcos_imprv_R_101_FPN",          ("α=0", "F-BCNet"), BCNET_1],  
    ["sotr_R101",                     ("α=0", "SOTR"),     SOTR_1],
    ["solov2_R50_3x",                 ("α=0", "SOLO"),     SOLO_1],
     
    ["mask_rcnn_R_50_FPN_1x",         ("α=1", "M-RCNN"),   MASK_1],  
    #["R_50_FPN_Supervised_6x",        ("α=1", "M-RCNN"),   MASK_1],
    #["R_50_FPN_Random_6x",            ("α=1", "rM-RCNN"),    MASK_1],
    #["mask_rcnn_R_50_FPN_100ep_LSJ",  ("α=1", "lM-RCNN"),  MASK_3],
    ["bmask_rcnn_r50_1x",             ("α=1", "BMask"),    MASK_1],  
    ["cascade_bmask_rcnn_r50_1x",     ("α=1", "C-BMask"),  CASK_1],
    ["cascade_mask_rcnn_R_50_FPN_1x", ("α=1", "Cascade"),  CASK_1],
    ["yolact-R50-FPN",                ("α=1", "YOLACT"), YOLACT_1],
    ["yolact-plus-R50-FPN",           ("α=1", "+YOLACT"),YOLACT_2],
    ["detr_r50",                      ("α=1", "DETR"),     DETR_1],
    ["fcos_imprv_R_101_FPN",          ("α=1", "F-BCNet"), BCNET_1],  
    ["sotr_R101",                     ("α=1", "SOTR"),     SOTR_1],
    ["solov2_R50_3x",                 ("α=1", "SOLO"),     SOLO_1]
]
R101_MODELS = [
    ["mask_rcnn_R_101_FPN_100ep_LSJ", ("α=0", "Mask R-CNN"), MASK_3],
    ["panoptic_fpn_R_101_dconv_cascade_gn_3x", ("α=0", "Cascade"), CASK_1],
    ["yolact-R101-FPN",               ("α=0", "YOLACT"),   YOLACT_1],
    ["yolact-plus-R101-FPN",          ("α=0", "+YOLACT"), YOLACT_2],
    ["detr_r101",                     ("α=0", "DETR"),       DETR_1],
    ["sotr_R101",                     ("α=0", "SOTR"),       SOTR_1], 
    ["solov2_R101_3x",                ("α=0", "SOLO"),       SOLO_1],

    ["mask_rcnn_R_101_FPN_100ep_LSJ", ("α=1", "Mask R-CNN"), MASK_3],
    ["panoptic_fpn_R_101_dconv_cascade_gn_3x", ("α=1", "Cascade"), CASK_1],
    ["yolact-R101-FPN",               ("α=1", "YOLACT"),   YOLACT_1],
    ["yolact-plus-R101-FPN",          ("α=1", "+YOLACT"), YOLACT_2],
    ["detr_r101",                     ("α=1", "DETR"),       DETR_1],
    ["sotr_R101",                     ("α=1", "SOTR"),       SOTR_1],
    ["solov2_R101_3x",                ("α=1", "SOLO"),       SOLO_1]    
]

print(len(R50_MODELS))
print(len(R101_MODELS))

20
14


In [9]:
def compare_frameworks(models: list, factors: list, colors: list, 
                       stats: list = ["AP","APs","APm","APl"], 
                       REL_ERROR:bool = True, title: str = "Title", iou_type: str = 'segm'):

    DATASETS = [STYLIZED_COCO, STYLIZED_COCO_OBJECTS, STYLIZED_COCO_BACKGROUND]
    plots = []
    for stat in stats:
        # prepare figure
        p = figure(x_range = FactorRange(*factors),
                   title=f"{'r'if REL_ERROR else ''}{stat}{(' ('+ iou_type + ')') if iou_type == 'bbox' else ''}",
                   plot_height=220, plot_width=300,
                   toolbar_location=None, tools="",output_backend='svg')

        # split 
        l  = len(models)//2
        factor_split = [factors[:l], factors[l:]]
        colors_split = [colors[:l],  colors[ l:]]
        alphas  = [0, -1] # a=0.0, a=1.0
        
        for a, f, c in zip(alphas, factor_split, colors_split):
                
            # prepare data
            sc,so,sb = [],[],[]
            for m in models[:l]:
                for s, dataset in zip([sc,so,sb], DATASETS):
                    y = SUMMARY[m][dataset]['1']['feature_space'][iou_type][stat][a]

                    if REL_ERROR:
                        # the uncorrupted coco dataset is alpha=0 in pixel space
                        coco_score = SUMMARY[m][dataset]['1']['pixel_space'][iou_type][stat][0]
                        y = y / coco_score
                    s.append(y)
            
            # plot data
            for y, gf, d in zip([sb, so, sc],[ p.square, p.triangle, p.circle],['s-background', 's-objects', 's-coco']):  
                    p.line(x=f, y=y, color='gray', line_width=1, line_alpha=1.0)
                    gf(    x=f, y=y, color='gray',       size=1, alpha=1, legend_label=d)
                    gf(    x=f, y=y, color=c,            size=5, alpha=1)
                    
        if stat == 'AP':
            p.legend.location = (7, 4)
            p.legend.label_text_font_size = "8px"
            p.legend.glyph_width, p.legend.glyph_height = 8, 8
            p.legend.margin, p.legend.padding = 7, 4
            p.legend.label_height = 4
        else:
            p.legend.visible = False

        # configure plot
        p.y_range.start = 0.0
        p.y_range.end   = 1.1
        p.x_range.range_padding    = 0.1 # left and right of x-range                  # default = 0.1
        p.x_range.factor_padding   = 0.1 # between highest level factors (factors)    # default = 0.0
        p.x_range.subgroup_padding = 1.2  # between middle  level factors (sub-groups) # default = 0.8
        p.x_range.group_padding    = 1.5  # between lowest  level factors (groups)     # default = 1.4
        p.xgrid.grid_line_color    = None
        p.yaxis.ticker=FixedTicker(ticks=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

        # a little hack since svg export doesn't align rotated xlabels correctly
        # create space but hide xlabels
        p.xaxis.major_label_orientation = math.pi/2
        p.xaxis.major_label_text_alpha = 0
        # fake labels as text which align correctly when rotated
        p.text(x=factors, y=[-0.08 for y in range(len(factors))], text=[f[1] for f in factors],
               text_align='right', angle=math.pi/2, text_font_size='10px', 
               x_offset=6, y_offset=1)
        
        p.xaxis.major_label_standoff = -2
        p.yaxis.axis_label_standoff = 0
        
        plots.append(p)
    
    # preprare grid plot with main title
    title = Div(text=f"<h3 style='margin-top: 2.5px'>{title} backbone by Framework</h3>", align='center')
    grid  = gridplot(plots, ncols=4, plot_width=270, plot_height=215, toolbar_location=None)
    #plot = column(title, grid) 
    plot = grid 
    
    # show plot
    show(plot)
    
    # for saving
    return plot

In [10]:
#for MODELS in [R50_MODELS, R101_MODELS]:
for MODELS in [R50_MODELS]:

    architecture = "ResNet-50"
    models, factors, colors = map(list,(zip(*MODELS)))
    stats = ['AP','APs','APm','APl']
    p     = compare_frameworks(models, factors, colors, stats=stats, REL_ERROR=True, title=architecture, iou_type='segm')
    #export_svg(p, filename=f"../plots/draft_v2/frameworks_{architecture}.svg")

In [9]:
BLUR_MODELS = [    
    ["mask_rcnn_R_50_DC5_3x",             ("a=0", "Mask R50 DC5 (36)")         ,MASK_1], 
    ["mask_rcnn_R_50_FPN_3x",             ("a=0", "Mask R50 FPN (36)")         ,MASK_1],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",      ("a=0", "Mask R50 FPN-LSJ (400)")    ,MASK_3],    
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("a=0", "Mask R50 DCN (36)")         ,MASK_1],
    ["mask_rcnn_R_101_DC5_3x",            ("a=0", "Mask R101 DC5 (36)")        ,MASK_1],
    ["mask_rcnn_R_101_FPN_3x",            ("a=0", "Mask R101 FPN (36)")        ,MASK_1],
    ["mask_rcnn_swin-T_3x",               ("a=0", "Swin-T (36)")               ,MASK_2],
    ["mask_rcnn_swin-S_3x",               ("a=0", "Swin-S (36)")               ,MASK_2],
    ["cascade_mask_rcnn_swin-B_3x",       ("a=0", "Swin-B (36)")       ,CASK_2],
    ["yolact-R50-FPN",                    ("a=0", "Yolact R50 FPN (53)")       ,YOLACT_1],
    ["yolact-R101-FPN",                   ("a=0", "Yolact R101 FPN (53)")      ,YOLACT_1],
    ["yolact-plus-R50-FPN",               ("a=0", "Yolact++ R50 DCN (53)")     ,YOLACT_2],
    ["yolact-plus-R101-FPN",              ("a=0", "Yolact++ R101 DCN (53)")    ,YOLACT_2],
    ["sotr_R101",                         ("a=0", "SOTR R101 FPN (20)")        ,SOTR_1], 
    ["sotr_R101_dcn",                     ("a=0", "SOTR R101 DCN (20)")        ,SOTR_1],
    ["solov2_R50_3x",                     ("a=0", "SOLOv2 R50 FPN (36)")       ,SOLO_1],
    ["solov2_R101_3x",                    ("a=0", "SOLOv2 R101 FPN (36)")      ,SOLO_1],
    
    ["mask_rcnn_R_50_DC5_3x",             ("a=1", "Mask R50 DC5 (36)")         ,MASK_1], 
    ["mask_rcnn_R_50_FPN_3x",             ("a=1", "Mask R50 FPN (36)")         ,MASK_1],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",      ("a=1", "Mask R50 FPN-LSJ (400)")    ,MASK_3],    
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("a=1", "Mask R50 DCN (36)")         ,MASK_1],
    ["mask_rcnn_R_101_DC5_3x",            ("a=1", "Mask R101 DC5 (36)")        ,MASK_1],
    ["mask_rcnn_R_101_FPN_3x",            ("a=1", "Mask R101 FPN (36)")        ,MASK_1],
    ["mask_rcnn_swin-T_3x",               ("a=1", "Swin-T (36)")               ,MASK_2],
    ["mask_rcnn_swin-S_3x",               ("a=1", "Swin-S (36)")               ,MASK_2],
    ["cascade_mask_rcnn_swin-B_3x",       ("a=1", "Swin-B (36)")       ,CASK_2],
    ["yolact-R50-FPN",                    ("a=1", "Yolact R50 FPN (53)")       ,YOLACT_1],
    ["yolact-R101-FPN",                   ("a=1", "Yolact R101 FPN (53)")      ,YOLACT_1],
    ["yolact-plus-R50-FPN",               ("a=1", "Yolact++ R50 DCN (53)")     ,YOLACT_2],
    ["yolact-plus-R101-FPN",              ("a=1", "Yolact++ R101 DCN (53)")    ,YOLACT_2],
    ["sotr_R101",                         ("a=1", "SOTR R101 FPN (20)")        ,SOTR_1], 
    ["sotr_R101_dcn",                     ("a=1", "SOTR R101 DCN (20)")        ,SOTR_1],
    ["solov2_R50_3x",                     ("a=1", "SOLOv2 R50 FPN (36)")       ,SOLO_1],
    ["solov2_R101_3x",                    ("a=1", "SOLOv2 R101 FPN (36)")      ,SOLO_1],
    
    ["mask_rcnn_R_50_DC5_3x",             ("blurred contour", "Mask R50 DC5 (36)")         ,MASK_1], 
    ["mask_rcnn_R_50_FPN_3x",             ("blurred contour", "Mask R50 FPN (36)")         ,MASK_1],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",      ("blurred contour", "Mask R50 FPN-LSJ (400)")    ,MASK_3],    
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("blurred contour", "Mask R50 DCN (36)")         ,MASK_1],
    ["mask_rcnn_R_101_DC5_3x",            ("blurred contour", "Mask R101 DC5 (36)")        ,MASK_1],
    ["mask_rcnn_R_101_FPN_3x",            ("blurred contour", "Mask R101 FPN (36)")        ,MASK_1],
    ["mask_rcnn_swin-T_3x",               ("blurred contour", "Swin-T (36)")               ,MASK_2],
    ["mask_rcnn_swin-S_3x",               ("blurred contour", "Swin-S (36)")               ,MASK_2],
    ["cascade_mask_rcnn_swin-B_3x",       ("blurred contour", "Swin-B (36)")               ,CASK_2],
    ["yolact-R50-FPN",                    ("blurred contour", "Yolact R50 FPN (53)")       ,YOLACT_1],
    ["yolact-R101-FPN",                   ("blurred contour", "Yolact R101 FPN (53)")      ,YOLACT_1],
    ["yolact-plus-R50-FPN",               ("blurred contour", "Yolact++ R50 DCN (53)")     ,YOLACT_2],
    ["yolact-plus-R101-FPN",              ("blurred contour", "Yolact++ R101 DCN (53)")    ,YOLACT_2],
    ["sotr_R101",                         ("blurred contour", "SOTR R101 FPN (20)")        ,SOTR_1], 
    ["sotr_R101_dcn",                     ("blurred contour", "SOTR R101 DCN (20)")        ,SOTR_1],
    ["solov2_R50_3x",                     ("blurred contour", "SOLOv2 R50 FPN (36)")       ,SOLO_1],
    ["solov2_R101_3x",                    ("blurred contour", "SOLOv2 R101 FPN (36)")      ,SOLO_1],
    
]
print(len(BLUR_MODELS))

51


In [10]:
SUMMARY['mask_rcnn_R_50_DC5_3x'].keys()


dict_keys(['coco_blur', 'sap_coco', 'sap_objects', 'sap_background', 'stylized_coco', 'stylized_coco_objects', 'stylized_coco_background'])

In [11]:
def compare_frameworks_blur(models: list, factors: list, colors: list, 
                       stats: list = ["AP","APs","APm","APl"], 
                       REL_ERROR:bool = True, title: str = "Title", iou_type: str = 'segm'):

    DATASETS = [STYLIZED_COCO, STYLIZED_COCO_OBJECTS, STYLIZED_COCO_BACKGROUND]
    plots = []
    for stat in stats:
        # prepare figure
        p = figure(x_range = FactorRange(*factors),
                   title=f"{'r'if REL_ERROR else ''}{stat}{(' ('+ iou_type + ')') if iou_type == 'bbox' else ''}",
                   plot_height=220, plot_width=300,
                   toolbar_location=None, tools="",output_backend='svg')

        # split 
        l  = len(models)//3
        factor_split = [factors[:l], factors[l:2*l], factors[2*l:]]
        colors_split = [colors[:l],  colors[ l:2*l], colors[2*l:]]
        model_split  = [models[:l],  models[ l:2*l], models[2*l:]]
        alphas  = [0, -1, 'sap'] # a=0.0, a=1.0
        
        for a, f, c, models, datasets in zip(alphas, factor_split, colors_split, model_split,
                                            [DATASETS, DATASETS, ['coco_blur']]):

            if datasets[0] == 'coco_blur':
                sb = []
                for m in models:
                             
                    y = SUMMARY[m]['coco_blur']['segm'][stat][-1]

                    if REL_ERROR:
                        # the uncorrupted coco dataset is alpha=0 in pixel space
                        coco_score = SUMMARY[m]['stylized_coco']['1']['pixel_space'][iou_type][stat][0]
                        y = y / coco_score
                        sb.append(y)
                # plot data
                for y, gf, d in zip([sb],[p.diamond],['blurred contour']):  
                        p.line(x=f, y=y, color='gray', line_width=1, line_alpha=1.0)
                        gf(    x=f, y=y, color='gray',       size=2, alpha=1, legend_label=d)
                        gf(    x=f, y=y, color=c,            size=10, alpha=1)
                           
            else:
                # prepare data
                sc,so,sb = [],[],[]
                for m in models:

                    for s, dataset in zip([sc,so,sb], datasets):

                        y = SUMMARY[m][dataset]['1']['feature_space'][iou_type][stat][a]

                        if REL_ERROR:
                            # the uncorrupted coco dataset is alpha=0 in pixel space
                            coco_score = SUMMARY[m]['stylized_coco']['1']['pixel_space'][iou_type][stat][0]
                            y = y / coco_score
                        s.append(y)

                # plot data
                for y, gf, d in zip([sb, so, sc],[ p.square, p.triangle, p.circle],['s-background', 's-objects', 's-coco']):  
                        p.line(x=f, y=y, color='gray', line_width=1, line_alpha=1.0)
                        gf(    x=f, y=y, color='gray',       size=1, alpha=1, legend_label=d)
                        gf(    x=f, y=y, color=c,            size=5, alpha=1)
                    
                    
                    
        if not stat == 'APs':
            p.legend.location = (15, 4)
            p.legend.label_text_font_size = '8px'
            p.legend.glyph_width, p.legend.glyph_height = 8,8
            p.legend.margin, p.legend.padding = 7, 4
            p.legend.label_height = 4
        else:
            p.legend.visible = False

        # configure plot
        p.y_range.start = 0.0
        p.y_range.end   = 1.1
        p.x_range.range_padding    = 0.1 # left and right of x-range                  # default = 0.1
        p.x_range.factor_padding   = 0.1 # between highest level factors (factors)    # default = 0.0
        p.x_range.subgroup_padding = 1.2  # between middle  level factors (sub-groups) # default = 0.8
        p.x_range.group_padding    = 1.5  # between lowest  level factors (groups)     # default = 1.4
        p.xgrid.grid_line_color    = None
        p.yaxis.ticker=FixedTicker(ticks=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

        # a little hack since svg export doesn't align rotated xlabels correctly
        # create space but hide xlabels
        p.xaxis.major_label_orientation = math.pi/2
        p.xaxis.major_label_text_alpha = 0
        # fake labels as text which align correctly when rotated
        p.text(x=factors, y=[-0.08 for y in range(len(factors))], text=[f[1] for f in factors],
               text_align='right', angle=math.pi/2, text_font_size='10px', 
               x_offset=6, y_offset=1)
        
        p.xaxis.major_label_standoff = -2
        p.yaxis.axis_label_standoff = 0
        
        plots.append(p)
    
    # preprare grid plot with main title
    title = Div(text=f"<h3 style='margin-top: 2.5px'>{title} backbone by Framework</h3>", align='center')
    grid  = gridplot(plots, ncols=4, plot_width=750, plot_height=300, toolbar_location=None)
    #plot = column(title, grid) 
    plot = grid 
    
    # show plot
    show(plot)
    
    # for saving
    return plot

In [14]:
for MODELS in [BLUR_MODELS]:
    architecture = "ResNet-50"
    models, factors, colors = map(list,(zip(*MODELS)))
    #for stats in ['AP','APs','APm','APl']:
    for stats in ['APl']:
        p     = compare_frameworks_blur(models, factors, colors, stats=[stats], REL_ERROR=True, title=architecture, iou_type='segm')
        #export_svg(p, filename=f"../plots/draft_v2/blur_comparison_{stats}.svg")

In [39]:
SAP_MODELS = [    
    ["mask_rcnn_R_50_DC5_3x",             ("a=0", "Mask R50 DC5 (36)")         ,MASK_1], 
    ["mask_rcnn_R_50_FPN_3x",             ("a=0", "Mask R50 FPN (36)")         ,MASK_1],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",      ("a=0", "Mask R50 FPN-LSJ (400)")    ,MASK_3],    
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("a=0", "Mask R50 DCN (36)")         ,MASK_1],
    ["mask_rcnn_R_101_DC5_3x",            ("a=0", "Mask R101 DC5 (36)")        ,MASK_1],
    ["mask_rcnn_R_101_FPN_3x",            ("a=0", "Mask R101 FPN (36)")        ,MASK_1],
    ["mask_rcnn_swin-T_3x",               ("a=0", "Swin-T (36)")               ,MASK_2],
    ["mask_rcnn_swin-S_3x",               ("a=0", "Swin-S (36)")               ,MASK_2],
    ["cascade_mask_rcnn_swin-B_3x",       ("a=0", "Swin-B (36)")       ,CASK_2],
    ["yolact-R50-FPN",                    ("a=0", "Yolact R50 FPN (53)")       ,YOLACT_1],
    ["yolact-R101-FPN",                   ("a=0", "Yolact R101 FPN (53)")      ,YOLACT_1],
    ["yolact-plus-R50-FPN",               ("a=0", "Yolact++ R50 DCN (53)")     ,YOLACT_2],
    ["yolact-plus-R101-FPN",              ("a=0", "Yolact++ R101 DCN (53)")    ,YOLACT_2],
    ["sotr_R101",                         ("a=0", "SOTR R101 FPN (20)")        ,SOTR_1], 
    ["sotr_R101_dcn",                     ("a=0", "SOTR R101 DCN (20)")        ,SOTR_1],
    ["solov2_R50_3x",                     ("a=0", "SOLOv2 R50 FPN (36)")       ,SOLO_1],
    ["solov2_R101_3x",                    ("a=0", "SOLOv2 R101 FPN (36)")      ,SOLO_1],
    
    ["mask_rcnn_R_50_DC5_3x",             ("a=1", "Mask R50 DC5 (36)")         ,MASK_1], 
    ["mask_rcnn_R_50_FPN_3x",             ("a=1", "Mask R50 FPN (36)")         ,MASK_1],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",      ("a=1", "Mask R50 FPN-LSJ (400)")    ,MASK_3],    
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("a=1", "Mask R50 DCN (36)")         ,MASK_1],
    ["mask_rcnn_R_101_DC5_3x",            ("a=1", "Mask R101 DC5 (36)")        ,MASK_1],
    ["mask_rcnn_R_101_FPN_3x",            ("a=1", "Mask R101 FPN (36)")        ,MASK_1],
    ["mask_rcnn_swin-T_3x",               ("a=1", "Swin-T (36)")               ,MASK_2],
    ["mask_rcnn_swin-S_3x",               ("a=1", "Swin-S (36)")               ,MASK_2],
    ["cascade_mask_rcnn_swin-B_3x",       ("a=1", "Swin-B (36)")       ,CASK_2],
    ["yolact-R50-FPN",                    ("a=1", "Yolact R50 FPN (53)")       ,YOLACT_1],
    ["yolact-R101-FPN",                   ("a=1", "Yolact R101 FPN (53)")      ,YOLACT_1],
    ["yolact-plus-R50-FPN",               ("a=1", "Yolact++ R50 DCN (53)")     ,YOLACT_2],
    ["yolact-plus-R101-FPN",              ("a=1", "Yolact++ R101 DCN (53)")    ,YOLACT_2],
    ["sotr_R101",                         ("a=1", "SOTR R101 FPN (20)")        ,SOTR_1], 
    ["sotr_R101_dcn",                     ("a=1", "SOTR R101 DCN (20)")        ,SOTR_1],
    ["solov2_R50_3x",                     ("a=1", "SOLOv2 R50 FPN (36)")       ,SOLO_1],
    ["solov2_R101_3x",                    ("a=1", "SOLOv2 R101 FPN (36)")      ,SOLO_1],
    
    ["mask_rcnn_R_50_DC5_3x",             ("SaP=0.2", "Mask R50 DC5 (36)")         ,MASK_1], 
    ["mask_rcnn_R_50_FPN_3x",             ("SaP=0.2", "Mask R50 FPN (36)")         ,MASK_1],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",      ("SaP=0.2", "Mask R50 FPN-LSJ (400)")    ,MASK_3],    
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("SaP=0.2", "Mask R50 DCN (36)")         ,MASK_1],
    ["mask_rcnn_R_101_DC5_3x",            ("SaP=0.2", "Mask R101 DC5 (36)")        ,MASK_1],
    ["mask_rcnn_R_101_FPN_3x",            ("SaP=0.2", "Mask R101 FPN (36)")        ,MASK_1],
    ["mask_rcnn_swin-T_3x",               ("SaP=0.2", "Swin-T (36)")               ,MASK_2],
    ["mask_rcnn_swin-S_3x",               ("SaP=0.2", "Swin-S (36)")               ,MASK_2],
    ["cascade_mask_rcnn_swin-B_3x",       ("SaP=0.2", "Swin-B (36)")       ,CASK_2],
    ["yolact-R50-FPN",                    ("SaP=0.2", "Yolact R50 FPN (53)")       ,YOLACT_1],
    ["yolact-R101-FPN",                   ("SaP=0.2", "Yolact R101 FPN (53)")      ,YOLACT_1],
    ["yolact-plus-R50-FPN",               ("SaP=0.2", "Yolact++ R50 DCN (53)")     ,YOLACT_2],
    ["yolact-plus-R101-FPN",              ("SaP=0.2", "Yolact++ R101 DCN (53)")    ,YOLACT_2],
    ["sotr_R101",                         ("SaP=0.2", "SOTR R101 FPN (20)")        ,SOTR_1], 
    ["sotr_R101_dcn",                     ("SaP=0.2", "SOTR R101 DCN (20)")        ,SOTR_1],
    ["solov2_R50_3x",                     ("SaP=0.2", "SOLOv2 R50 FPN (36)")       ,SOLO_1],
    ["solov2_R101_3x",                    ("SaP=0.2", "SOLOv2 R101 FPN (36)")      ,SOLO_1],
    
]
print(len(SAP_MODELS))

51


In [44]:
def compare_frameworks_sap(models: list, factors: list, colors: list, 
                       stats: list = ["AP","APs","APm","APl"], 
                       REL_ERROR:bool = True, title: str = "Title", iou_type: str = 'segm'):

    DATASETS = [STYLIZED_COCO, STYLIZED_COCO_OBJECTS, STYLIZED_COCO_BACKGROUND]
    plots = []
    for stat in stats:
        # prepare figure
        p = figure(x_range = FactorRange(*factors),
                   title=f"{'r'if REL_ERROR else ''}{stat}{(' ('+ iou_type + ')') if iou_type == 'bbox' else ''}",
                   plot_height=220, plot_width=300,
                   toolbar_location=None, tools="",output_backend='svg')

        # split 
        l  = len(models)//3
        factor_split = [factors[:l], factors[l:2*l], factors[2*l:]]
        colors_split = [colors[:l],  colors[ l:2*l], colors[2*l:]]
        model_split  = [models[:l],  models[ l:2*l], models[2*l:]]
        alphas  = [0, -1, 'sap'] # a=0.0, a=1.0
        
        for a, f, c, models, datasets in zip(alphas, factor_split, colors_split, model_split,
                                            [DATASETS, DATASETS, ['sap_coco', 'sap_objects', 'sap_background']]):

            # prepare data
            sc,so,sb = [],[],[]
            for m in models:
                for s, dataset in zip([sc,so,sb], datasets):
                    
                    if 'sap' in dataset:
                        y = SUMMARY[m][dataset]['segm'][stat][-1]
                        #print(m, y, SUMMARY[m]['stylized_coco']['1']['pixel_space'][iou_type][stat][0])
                    else:
                        y = SUMMARY[m][dataset]['1']['feature_space'][iou_type][stat][a]

                    if REL_ERROR:
                        # the uncorrupted coco dataset is alpha=0 in pixel space
                        coco_score = SUMMARY[m]['stylized_coco']['1']['pixel_space'][iou_type][stat][0]
                        y = y / coco_score
                    s.append(y)
            
            # plot data
            for y, gf, d in zip([sb, so, sc],[ p.square, p.triangle, p.circle],['s-background', 's-objects', 's-coco']):  
                    p.line(x=f, y=y, color='gray', line_width=1, line_alpha=1.0)
                    gf(    x=f, y=y, color='gray',       size=1, alpha=1, legend_label=d)
                    gf(    x=f, y=y, color=c,            size=5, alpha=1)
                    
        if not stat == 'APs':
            p.legend.location = (15, 4)
            p.legend.label_text_font_size = '8px'
            p.legend.glyph_width, p.legend.glyph_height = 8,8
            p.legend.margin, p.legend.padding = 7, 4
            p.legend.label_height = 4
        else:
            p.legend.visible = False

        # configure plot
        p.y_range.start = 0.0
        p.y_range.end   = 1.1
        p.x_range.range_padding    = 0.1 # left and right of x-range                  # default = 0.1
        p.x_range.factor_padding   = 0.1 # between highest level factors (factors)    # default = 0.0
        p.x_range.subgroup_padding = 1.2  # between middle  level factors (sub-groups) # default = 0.8
        p.x_range.group_padding    = 1.5  # between lowest  level factors (groups)     # default = 1.4
        p.xgrid.grid_line_color    = None
        p.yaxis.ticker=FixedTicker(ticks=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

        # a little hack since svg export doesn't align rotated xlabels correctly
        # create space but hide xlabels
        p.xaxis.major_label_orientation = math.pi/2
        p.xaxis.major_label_text_alpha = 0
        # fake labels as text which align correctly when rotated
        p.text(x=factors, y=[-0.08 for y in range(len(factors))], text=[f[1] for f in factors],
               text_align='right', angle=math.pi/2, text_font_size='10px', 
               x_offset=6, y_offset=1)
        
        p.xaxis.major_label_standoff = -2
        p.yaxis.axis_label_standoff = 0
        
        plots.append(p)
    
    # preprare grid plot with main title
    title = Div(text=f"<h3 style='margin-top: 2.5px'>{title} backbone by Framework</h3>", align='center')
    grid  = gridplot(plots, ncols=4, plot_width=750, plot_height=300, toolbar_location=None)
    #plot = column(title, grid) 
    plot = grid 
    
    # show plot
    show(plot)
    
    # for saving
    return plot

In [45]:
for MODELS in [SAP_MODELS]:
    architecture = "ResNet-50"
    models, factors, colors = map(list,(zip(*MODELS)))
    for stats in ['AP','APs','APm','APl']:
        p     = compare_frameworks_sap(models, factors, colors, stats=[stats], REL_ERROR=True, title=architecture, iou_type='segm')
        #export_svg(p, filename=f"../plots/draft_v2/sap_comparison_{stats}.svg")

## Compare backbones and necks

In [18]:
# Compare FPN Neck by Backbone
BB_A0 = [
    ["mask_rcnn_R_50_FPN_3x",         ("α=0", "Mask R-CNN", "R50") ,       MASK_1],
    ["mask_rcnn_R_101_FPN_3x",        ("α=0", "Mask R-CNN", "R101"),       MASK_1],
    ["mask_rcnn_R_50_FPN_100ep_LSJ",  ("α=0", "Mask R-CNN", "*R50"),  MASK_3],
    ["mask_rcnn_R_101_FPN_100ep_LSJ", ("α=0", "Mask R-CNN", "*R101"), MASK_3],

    ["mask_rcnn_X_101_32x8d_FPN_3x",            ("α=0", "Mask R-CNN", "X101"), MASK_1],
    ["mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ", ("α=0", "Mask R-CNN", "*RegX"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ", ("α=0", "Mask R-CNN", "*RegY"), MASK_3],
    
    ["mask_rcnn_swin-T_3x",         ("α=0", "Mask R-CNN", "Swin-T"), MASK_2],
    ["mask_rcnn_swin-S_3x",         ("α=0", "Mask R-CNN", "Swin-S"), MASK_2],
    
    ["bmask_rcnn_r50_1x",             ("α=0", "BM",    "R50"),  MASK_1],  
    ["bmask_rcnn_r101_1x",            ("α=0", "BM",   "R101"),  MASK_1],  
    ["cascade_bmask_rcnn_r50_1x",     ("α=0", "C-BM",  "R50"),  CASK_1],
    ["cascade_bmask_rcnn_r101_1x",    ("α=0", "C-BM", "R101"),  CASK_1],
    
    ["cascade_mask_rcnn_R_50_FPN_3x", ("α=0","Cascade", "R50"),    CASK_1],
    ["cascade_mask_rcnn_swin-T_3x",   ("α=0","Cascade", "Swin-T"), CASK_2],
    ["cascade_mask_rcnn_swin-S_3x",   ("α=0","Cascade", "Swin-S"), CASK_2],
    ["cascade_mask_rcnn_swin-B_3x",   ("α=0","Cascade", "Swin-B"), CASK_2],

    ["yolact-R50-FPN",       ("α=0", "Yolact", "R50"),  YOLACT_1],
    ["yolact-R101-FPN",      ("α=0", "Yolact", "R101"), YOLACT_1],
    ["yolact-plus-R50-FPN",       ("α=0", "Yolact", "++R50"),  YOLACT_2],
    ["yolact-plus-R101-FPN",      ("α=0", "Yolact", "++R101"), YOLACT_2],

    ["detr_r50",             ("α=0", "DETR", "R50") , DETR_1], 
    ["detr_r101",            ("α=0", "DETR", "R101"), DETR_1],
    ["solov2_R50_3x",        ("α=0", "SOLO", "R50"),  SOLO_1],    
    ["solov2_R101_3x",       ("α=0", "SOLO", "R101"), SOLO_1]
]
BB_A1 = [
    ["mask_rcnn_R_50_FPN_3x",         ("α=1", "Mask R-CNN","R50") ,       MASK_1],
    ["mask_rcnn_R_101_FPN_3x",        ("α=1", "Mask R-CNN", "R101"),      MASK_1],
    ["mask_rcnn_R_50_FPN_100ep_LSJ",  ("α=1", "Mask R-CNN","*R50"),  MASK_3],
    ["mask_rcnn_R_101_FPN_100ep_LSJ", ("α=1", "Mask R-CNN","*R101"), MASK_3],

    ["mask_rcnn_X_101_32x8d_FPN_3x",            ("α=1", "Mask R-CNN", "X101"), MASK_1],
    ["mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ", ("α=1", "Mask R-CNN", "*RegX"), MASK_3],
    ["mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ", ("α=1", "Mask R-CNN", "*RegY"), MASK_3],
    
    ["mask_rcnn_swin-T_3x",         ("α=1", "Mask R-CNN", "Swin-T"), MASK_2],
    ["mask_rcnn_swin-S_3x",         ("α=1", "Mask R-CNN", "Swin-S"), MASK_2],
    
    ["bmask_rcnn_r50_1x",             ("α=1", "BM",    "R50"),  MASK_1],  
    ["bmask_rcnn_r101_1x",            ("α=1", "BM",   "R101"),  MASK_1],  
    ["cascade_bmask_rcnn_r50_1x",     ("α=1", "C-BM",  "R50"),  CASK_1],
    ["cascade_bmask_rcnn_r101_1x",    ("α=1", "C-BM", "R101"),  CASK_1],

    ["cascade_mask_rcnn_R_50_FPN_3x", ("α=1", "Cascade", "R50"),    CASK_1],
    ["cascade_mask_rcnn_swin-T_3x",   ("α=1", "Cascade", "Swin-T"), CASK_2],
    ["cascade_mask_rcnn_swin-S_3x",   ("α=1", "Cascade", "Swin-S"), CASK_2],
    ["cascade_mask_rcnn_swin-B_3x",   ("α=1", "Cascade", "Swin-B"), CASK_2],

    ["yolact-R50-FPN",       ("α=1", "Yolact", "R50"),  YOLACT_1],
    ["yolact-R101-FPN",      ("α=1", "Yolact", "R101"), YOLACT_1],
    ["yolact-plus-R50-FPN",       ("α=1", "Yolact", "++R50"),  YOLACT_2],
    ["yolact-plus-R101-FPN",      ("α=1", "Yolact", "++R101"), YOLACT_2],

    ["detr_r50",             ("α=1", "DETR", "R50") , DETR_1], 
    ["detr_r101",            ("α=1", "DETR", "R101"), DETR_1],
    ["solov2_R50_3x",        ("α=1", "SOLO", "R50"),  SOLO_1],    
    ["solov2_R101_3x",       ("α=1", "SOLO", "R101"), SOLO_1]
]

In [19]:
def compare_backbone(models, factors, colors, model_pairs, plot_width, plot_height,
                     stats=['AP','APs','APm','APl'], grid_cols=2, REL_ERROR=True, 
                     subgroup_rotate=False, mark_dcn=False, ssl=False, iou_type='segm'):

    plots = []
    for stat in stats:
        
        # prepare plot
        p = figure(x_range = FactorRange(*factors), y_range=[0, 1.1],
                   title=f"{'r'if REL_ERROR else ''}{stat}{(' ('+ iou_type + ')') if iou_type == 'bbox' else ''}",
                   plot_height=300, plot_width=580,
                   toolbar_location=None, tools="",output_backend='svg')

        # prepare data
        rs = None
        for dataset, gf, label in zip([STYLIZED_COCO_BACKGROUND, STYLIZED_COCO_OBJECTS, STYLIZED_COCO],
                                      [p.square,                 p.triangle,            p.circle], 
                                      ['s-background',          's-objects',            's-coco']):

            ys = []
            for alpha_index in [0, -1]: # alpha=0 and alpha=1.0
                for m in models[:len(models)//2]:

                    if iou_type == 'segm' and 'yolo' in m:
                        iou = 'bbox'
                    else:
                        iou = iou_type
                    y = SUMMARY[m][dataset]['1']['feature_space'][iou][stat][alpha_index]
                    if REL_ERROR:
                        # the uncorrupted coco dataset is alpha=0 in pixel space
                        coco_score = SUMMARY[m][dataset]['1']['pixel_space'][iou][stat][0]
                        y = y / coco_score
                    ys.append(y)

            # connect model pairs with a line
            # a=0
            for p1,p2 in model_pairs:
                p.line(factors[p1:p2], ys[p1:p2], line_color='gray', line_alpha=1)
            # a=1
            for p1,p2 in model_pairs:
                p1,p2 = p1+len(models)//2, p2+len(models)//2
                p.line(factors[p1:p2], ys[p1:p2], line_color='gray', line_alpha=1)

            # plot fake data for legend and actual data
            gf(x=factors, y=ys, color='gray', size=1, alpha=1, legend_label=label)
            gf(x=factors, y=ys, color=colors, size=5, alpha=1)

        # plot legend
        if stat == 'AP':
            p.legend.location = (7, 4)
            p.legend.label_text_font_size = "9px"
            p.legend.glyph_width, p.legend.glyph_height = 8, 8
            p.legend.margin, p.legend.padding = 7, 4
            p.legend.label_height = 5
            p.legend.background_fill_alpha = 1

            if mark_dcn:
                
                rm  = p.circle(x=0, y=0, color=MASK_1,   size=0.01, alpha=1)
                rc  = p.circle(x=0, y=0, color=CASK_1, size=0.01, alpha=1)
                rd  = p.circle(x=0, y=0, color=DETR_1, size=0.01, alpha=1)

                rs  = p.circle(x=0, y=0, color=SOTR_1, size=0.01, alpha=1)
                rya = p.circle(x=0, y=0, color=YOLACT_1, size=0.01, alpha=1)
                ryo = p.circle(x=0, y=0, color=YOLO_1, size=0.01, alpha=1)

                
                for x_pos, items in zip([95,175],[[LegendItem(label='Mask R-CNN', renderers=[rm]),
                                                  LegendItem(label='DETR', renderers=[rd]),
                                                  LegendItem(label='Yolact(++)',   renderers=[rya])],
                                                 [LegendItem(label='Cascade', renderers=[rc]),
                                                  LegendItem(label='SOTR', renderers=[rs]),
                                                  LegendItem(label='Yolo', renderers=[ryo])]]):
                     p.add_layout(Legend(items=items, location=(x_pos, 4),
                                         label_height=5, glyph_height=8, glyph_width=8,
                                         margin=7, padding=4, label_text_font_size="9px"
                                        )
                                 )
        else:
            p.legend.visible = False
            
        sgp = p.x_range.subgroup_padding
        gp  = p.x_range.group_padding
        if mark_dcn:
            
            p.add_layout(BoxAnnotation(left=3, right=4, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))
            p.add_layout(BoxAnnotation(left=7, right=8, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))

            p.add_layout(BoxAnnotation(left=11+sgp, right=12+sgp, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))
            p.add_layout(BoxAnnotation(left=13+sgp, right=14+sgp, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))
            p.add_layout(BoxAnnotation(left=15+sgp, right=16+sgp, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))

            p.add_layout(BoxAnnotation(left=17+3*sgp, right=18+3*sgp, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))

            p.add_layout(BoxAnnotation(left=25+gp+4*sgp, right=26+gp+4*sgp, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))
            p.add_layout(BoxAnnotation(left=29+gp+4*sgp, right=30+gp+4*sgp, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))

            p.add_layout(BoxAnnotation(left=33+gp+5*sgp, right=34+gp+5*sgp, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))        
            p.add_layout(BoxAnnotation(left=35+gp+5*sgp, right=36+gp+5*sgp, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))            
            p.add_layout(BoxAnnotation(left=37+gp+5*sgp, right=38+gp+5*sgp, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))

            p.add_layout(BoxAnnotation(left=39+gp+7*sgp, right=40+gp+7*sgp, fill_color='gray', fill_alpha=0.1, line_alpha=0.1))
            
        # style plot
        p.xaxis.major_label_orientation = math.pi/2    
        p.xaxis.major_label_text_alpha  = 0
        # fake labels as text which align correctly when rotated
        
        
        text_font_style = ['normal' for f in factors]
        ft = [f[2] for f in factors]
        if mark_dcn:
            for i,f in enumerate(ft):
                if 'DCN' in f: text_font_style[i] = 'bold'
                for r in ["c-", "y-", "d-", "s-"]:
                    if r in f: 
                        ft[i] = f.replace(r,"")
                        continue

        p.text(x=factors, y=[-0.08 for y in range(len(factors))], text=ft,
               text_align='right', angle=math.pi/2, text_font_size='10px', 
               x_offset=6, y_offset=1, text_font_style=text_font_style)
        
        p.xaxis.major_label_standoff = -1
        p.yaxis.axis_label_standoff = 0
        
        p.xgrid.grid_line_color    = None
        p.xaxis.subgroup_text_font_size = '10px'
        if subgroup_rotate:
            p.xaxis.subgroup_label_orientation = math.pi/2
        p.yaxis.ticker=FixedTicker(ticks=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
        plots.append(p)
        
        p.x_range.range_padding = 0.03 # left and right of x-range                  # default = 0.1
    
    grid = gridplot(plots, ncols=2, plot_width=plot_width, plot_height=plot_height, toolbar_location=None)
    show(grid)
    return grid

In [20]:
MODELS = []
MODELS.extend(BB_A0)
MODELS.extend(BB_A1)
models, factors, colors = map(list,(zip(*MODELS)))
model_pairs = [(0,2),(2,4),(5,7),(7,9),(9,11),(11,13), (14,17),(17,19),(19,21),(21,23), (23,25)]

p = compare_backbone(models, factors, colors, model_pairs, plot_width=680, plot_height=245, stats=['AP','APs'], 
                     iou_type='segm')

#export_svg(p, filename=f"../plots/draft_v2/compare_backends.svg")

In [21]:
NE_A0 = [
    ["mask_rcnn_R_50_FPN_3x",             ("α=0", "R50", "FPN"), MASK_1],
    ["mask_rcnn_R_50_C4_3x",              ("α=0", "R50", "C4") , MASK_1],
    ["mask_rcnn_R_50_DC5_3x",             ("α=0", "R50", "DC5"), MASK_1],
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("α=0", "R50", "DCN"), MASK_1],

    ["detr_r50",             ("α=0", "R50", "d-FPN") , DETR_1],
    ["detr_r50_dc5",         ("α=0", "R50", "d-DC5") , DETR_1], 
    
    ["yolact-R50-FPN",       ("α=0", "R50", "y-FPN"),  YOLACT_1],
    ["yolact-plus-R50-FPN",  ("α=0", "R50", "y-DCN"),  YOLACT_2],
    
    ["mask_rcnn_R_101_FPN_3x", ("α=0", "R101", "FPN"), MASK_1],
    ["mask_rcnn_R_101_C4_3x",  ("α=0", "R101", "C4"),  MASK_1],
    ["mask_rcnn_R_101_DC5_3x", ("α=0", "R101", "DC5"), MASK_1],
    ["panoptic_fpn_R_101_dconv_cascade_gn_3x", ("α=0", "R101", "c-DCN"), CASK_1],    
    
    ["yolact-R101-FPN",      ("α=0", "R101", "y-FPN"), YOLACT_1],
    ["yolact-plus-R101-FPN", ("α=0", "R101", "y-DCN"), YOLACT_2],
    ["sotr_R101",            ("α=0", "R101", "s-FPN") , SOTR_1],
    ["sotr_R101_dcn",        ("α=0", "R101", "s-DCN") , SOTR_1], 


    ["mask_rcnn_X_101_32x8d_FPN_3x", ("α=0", "X101", "FPN"), MASK_1],
    ["cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv", ("α=0", "X151", "c-DCN"), CASK_1],
    
    ["yolov4",     ("α=0", "CD53", "PAN"),     YOLO_1],
    ["yolov4-csp", ("α=0", "CD53", "CPAN"),    YOLO_1],
    ["yolov4-p5",  ("α=0", "CD53", "p5-CPAN"), YOLO_1],
    ["yolov4-p6",  ("α=0", "CD53", "p6-CPAN"), YOLO_1]
]
NE_A1 = [
    ["mask_rcnn_R_50_FPN_3x",             ("α=1", "R50", "FPN"), MASK_1],
    ["mask_rcnn_R_50_C4_3x",              ("α=1", "R50", "C4") , MASK_1],
    ["mask_rcnn_R_50_DC5_3x",             ("α=1", "R50", "DC5"), MASK_1],
    ["mask_rcnn_R_50_FPN_3x_dconv_c3-c5", ("α=1", "R50", "DCN"), MASK_1],

    ["detr_r50",             ("α=1", "R50", "d-FPN") , DETR_1],
    ["detr_r50_dc5",         ("α=1", "R50", "d-DC5") , DETR_1], 
    
    ["yolact-R50-FPN",       ("α=1", "R50", "y-FPN"),  YOLACT_1],
    ["yolact-plus-R50-FPN",  ("α=1", "R50", "y-DCN"),  YOLACT_2],
    
    ["mask_rcnn_R_101_FPN_3x", ("α=1", "R101", "FPN"), MASK_1],
    ["mask_rcnn_R_101_C4_3x",  ("α=1", "R101", "C4"),  MASK_1],
    ["mask_rcnn_R_101_DC5_3x", ("α=1", "R101", "DC5"), MASK_1],
    ["panoptic_fpn_R_101_dconv_cascade_gn_3x", ("α=1", "R101", "c-DCN"), CASK_1],    
    
    ["yolact-R101-FPN",      ("α=1", "R101", "y-FPN"), YOLACT_1],
    ["yolact-plus-R101-FPN", ("α=1", "R101", "y-DCN"), YOLACT_2],
    ["sotr_R101",            ("α=1", "R101", "s-FPN") , SOTR_1],
    ["sotr_R101_dcn",        ("α=1", "R101", "s-DCN") , SOTR_1], 


    ["mask_rcnn_X_101_32x8d_FPN_3x", ("α=1", "X101", "FPN"), MASK_1],
    ["cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv", ("α=1", "X151", "c-DCN"), CASK_1],
    
    ["yolov4",     ("α=1", "CD53", "PAN"),     YOLO_1],
    ["yolov4-csp", ("α=1", "CD53", "CPAN"),    YOLO_1],
    ["yolov4-p5",  ("α=1", "CD53", "p5-CPAN"), YOLO_1],
    ["yolov4-p6",  ("α=1", "CD53", "p6-CPAN"), YOLO_1]
]

In [22]:
MODELS = []
MODELS.extend(NE_A0)
MODELS.extend(NE_A1)
models, factors, colors = map(list,(zip(*MODELS)))
model_pairs = [(0,4),(4,6),(6,8),(8,11),(12,14),(14,16),(18,22)]

p = compare_backbone(models, factors, colors, model_pairs, plot_width=550, plot_height=265, stats=['AP', 'APs'],
                     subgroup_rotate=True, mark_dcn=True, iou_type='segm')
#export_svg(p, filename=f"../plots/draft_v1/compare_necks_bbox.svg")

NameError: name 'BoxAnnotation' is not defined

In [ ]:
PT0 = [
    ["mask_rcnn_R_50_FPN_1x",        ("α=0", "IN", "R50 (12)"),  MASK_1],
    ["mask_rcnn_R_50_FPN_3x",        ("α=0", "IN", "R50 (36)"),  MASK_1],    
    ["R_50_FPN_Supervised_6x",       ("α=0", "IN", "R50 (72)"),  MASK_1],    
    #["mask_rcnn_R_101_FPN_3x",       ("α=0", "IN", "R101 (36)"), MASK_1],
    #["mask_rcnn_X_101_32x8d_FPN_3x", ("α=0", "IN", "X101 (36)"), MASK_1],   
    
    ["R_50_FPN_Random_6x",  ("α=0", "R", "R50 (72)"), MASK_3],
    
    ["mask_rcnn_R_50_FPN_100ep_LSJ",  ("α=0", "LSJ", "R50 (100)"), MASK_3],
    ["mask_rcnn_R_50_FPN_200ep_LSJ",  ("α=0", "LSJ", "R50 (200)"), MASK_3],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",  ("α=0", "LSJ", "R50 (400)"), MASK_3],
    #["mask_rcnn_R_101_FPN_100ep_LSJ", ("α=0", "LSJ", "R101 (100)"), MASK_3],
    #["mask_rcnn_R_101_FPN_200ep_LSJ", ("α=0", "LSJ", "R101 (200)"), MASK_3],
    #["mask_rcnn_R_101_FPN_400ep_LSJ", ("α=0", "LSJ", "R110 (400)"), MASK_3],
    #["mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ", ("α=0", "LSJ", "RegX (100)"), MASK_3],
    #["mask_rcnn_regnetx_4gf_dds_FPN_200ep_LSJ", ("α=0", "LSJ", "RegX (200)"), MASK_3],
    #["mask_rcnn_regnetx_4gf_dds_FPN_400ep_LSJ", ("α=0", "LSJ", "RegX (400)"), MASK_3],
    #["mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ", ("α=0", "LSJ", "RegY (100)"), MASK_3],
    #["mask_rcnn_regnety_4gf_dds_FPN_200ep_LSJ", ("α=0", "LSJ", "RegY (200)"), MASK_3],
    #["mask_rcnn_regnety_4gf_dds_FPN_400ep_LSJ", ("α=0", "LSJ", "RegY (400)"), MASK_3],

    ["R_50_FPN_InstDis_1x", ("α=0", "SSL", "InstDis (12)"), MASK_4],
    ["R_50_FPN_InstDis_2x", ("α=0", "SSL", "InstDis (24)"), MASK_4],
    ["R_50_FPN_PIRL_1x",    ("α=0", "SSL", "PIRL (12)"), MASK_4],
    ["R_50_FPN_PIRL_2x",    ("α=0", "SSL", "PIRL (24)"), MASK_4],
    ["R_50_FPN_MoCo_v1_1x", ("α=0", "SSL", "MoCo-v1 (12)"), MASK_4],
    ["R_50_FPN_MoCo_v1_2x", ("α=0", "SSL", "MoCo-v1 (24)"), MASK_4],
    ["R_50_FPN_MoCo_v2_2x", ("α=0", "SSL", "MoCo-v2 (24)"), MASK_4],
    ["R_50_FPN_InfoMin_1x", ("α=0", "SSL", "InfoMin (12)"), MASK_4],
    ["R_50_FPN_InfoMin_2x", ("α=0", "SSL", "InfoMin (24)"), MASK_4],
    ["R_50_FPN_InfoMin_6x", ("α=0", "SSL", "InfoMin (72)"), MASK_4],
]
PT1 =[
    ["mask_rcnn_R_50_FPN_1x",        ("α=1", "IN", "R50 (12)"),  MASK_1],
    ["mask_rcnn_R_50_FPN_3x",        ("α=1", "IN", "R50 (36)"),  MASK_1],    
    ["R_50_FPN_Supervised_6x",       ("α=1", "IN", "R50 (72)"),  MASK_1],    
    #["mask_rcnn_R_101_FPN_3x",       ("α=1", "IN", "R101 (36)"), MASK_1],
    #["mask_rcnn_X_101_32x8d_FPN_3x", ("α=1", "IN", "X101 (36)"), MASK_1],   
    
    ["R_50_FPN_Random_6x",  ("α=1", "R", "R50 (72)"), MASK_3],    

    ["mask_rcnn_R_50_FPN_100ep_LSJ",  ("α=1", "LSJ", "R50 (100)"), MASK_3],
    ["mask_rcnn_R_50_FPN_200ep_LSJ",  ("α=1", "LSJ", "R50 (200)"), MASK_3],
    ["mask_rcnn_R_50_FPN_400ep_LSJ",  ("α=1", "LSJ", "R50 (400)"), MASK_3],
    #["mask_rcnn_R_101_FPN_100ep_LSJ", ("α=1", "LSJ", "R101 (100)"), MASK_3],
    #["mask_rcnn_R_101_FPN_200ep_LSJ", ("α=1", "LSJ", "R101 (200)"), MASK_3],
    #["mask_rcnn_R_101_FPN_400ep_LSJ", ("α=1", "LSJ", "R110 (400)"), MASK_3],
    #["mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ", ("α=1", "LSJ", "RegX (100)"), MASK_3],
    #["mask_rcnn_regnetx_4gf_dds_FPN_200ep_LSJ", ("α=1", "LSJ", "RegX (200)"), MASK_3],
    #["mask_rcnn_regnetx_4gf_dds_FPN_400ep_LSJ", ("α=1", "LSJ", "RegX (400)"), MASK_3],
    #["mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ", ("α=1", "LSJ", "RegY (100)"), MASK_3],
    #["mask_rcnn_regnety_4gf_dds_FPN_200ep_LSJ", ("α=1", "LSJ", "RegY (200)"), MASK_3],
    #["mask_rcnn_regnety_4gf_dds_FPN_400ep_LSJ", ("α=1", "LSJ", "RegY (400)"), MASK_3],
    
    ["R_50_FPN_InstDis_1x", ("α=1", "SSL", "InstDis (12)"), MASK_4],
    ["R_50_FPN_InstDis_2x", ("α=1", "SSL", "InstDis (24)"), MASK_4],
    ["R_50_FPN_PIRL_1x",    ("α=1", "SSL", "PIRL (12)"), MASK_4],
    ["R_50_FPN_PIRL_2x",    ("α=1", "SSL", "PIRL (24)"), MASK_4],
    ["R_50_FPN_MoCo_v1_1x", ("α=1", "SSL", "MoCo-v1 (12)"), MASK_4],
    ["R_50_FPN_MoCo_v1_2x", ("α=1", "SSL", "MoCo-v1 (24)"), MASK_4],
    ["R_50_FPN_MoCo_v2_2x", ("α=1", "SSL", "MoCo-v2 (24)"), MASK_4],
    ["R_50_FPN_InfoMin_1x", ("α=1", "SSL", "InfoMin (12)"), MASK_4],
    ["R_50_FPN_InfoMin_2x", ("α=1", "SSL", "InfoMin (24)"), MASK_4],
    ["R_50_FPN_InfoMin_6x", ("α=1", "SSL", "InfoMin (72)"), MASK_4],
]  

In [ ]:
MODELS = []
MODELS.extend(PT0)
MODELS.extend(PT1)
models, factors, colors = map(list,(zip(*MODELS)))
model_pairs = [(0,3),(4,7),(7,9),(9,11),(11,13),(14,17)]

p = compare_backbone(models, factors, colors, model_pairs, plot_width=420, plot_height=275,grid_cols=1, 
                     stats=['AP'], iou_type='segm')
#export_svg(p, filename=f"../plots/draft_v1/compare_pretraining_augmentation.svg")

### Compare models by group :)

In [8]:
MASK = ["Mask R-CNN", MASK_2, [
    "mask_rcnn_R_50_C4_1x",
    "mask_rcnn_R_50_DC5_1x",
    "mask_rcnn_R_50_FPN_1x",
    "mask_rcnn_R_50_C4_3x",
    "mask_rcnn_R_50_DC5_3x",
    "mask_rcnn_R_50_FPN_3x",
    "R_50_FPN_Supervised_6x",
    "mask_rcnn_R_101_C4_3x",
    "mask_rcnn_R_101_DC5_3x",
    "mask_rcnn_R_101_FPN_3x",
    "mask_rcnn_X_101_32x8d_FPN_3x",
    "mask_rcnn_R_50_FPN_1x_dconv_c3-c5",
    "mask_rcnn_R_50_FPN_3x_dconv_c3-c5",
    "mask_rcnn_R_50_FPN_100ep_LSJ",
    "mask_rcnn_R_50_FPN_200ep_LSJ",
    "mask_rcnn_R_50_FPN_400ep_LSJ",
    "mask_rcnn_R_101_FPN_100ep_LSJ",
    "mask_rcnn_R_101_FPN_200ep_LSJ",
    "mask_rcnn_R_101_FPN_400ep_LSJ",
    "mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ",
    "mask_rcnn_regnetx_4gf_dds_FPN_200ep_LSJ",
    "mask_rcnn_regnetx_4gf_dds_FPN_400ep_LSJ",
    "mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ",
    "mask_rcnn_regnety_4gf_dds_FPN_200ep_LSJ",
    "mask_rcnn_regnety_4gf_dds_FPN_400ep_LSJ",
    "R_50_FPN_InstDis_1x",
    "R_50_FPN_InstDis_2x",
    "R_50_FPN_PIRL_1x",
    "R_50_FPN_PIRL_2x",
    "R_50_FPN_MoCo_v1_1x",
    "R_50_FPN_MoCo_v1_2x",
    "R_50_FPN_MoCo_v2_2x",
    "R_50_FPN_InfoMin_1x",
    "R_50_FPN_InfoMin_2x",
    "R_50_FPN_InfoMin_6x"]
]
SWIN = ["Swin", CASK_2, [
    "mask_rcnn_swin-T_1x",
    "mask_rcnn_swin-T_3x",
    "mask_rcnn_swin-S_3x",
    "cascade_mask_rcnn_swin-T_1x",
    "cascade_mask_rcnn_swin-T_3x",
    "cascade_mask_rcnn_swin-S_3x",
    "cascade_mask_rcnn_swin-B_3x"]
]
CASCADE = ["Cascade", CASK_1, [
    "cascade_mask_rcnn_R_50_FPN_1x",
    "cascade_mask_rcnn_R_50_FPN_3x",
    "panoptic_fpn_R_101_dconv_cascade_gn_3x",
    "cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv"]
]
YOLO = ["Yolo", YOLO_1, [
    "yolov3",
    "yolov4",
    "yolov4-csp",
    "yolov4-p5",
    "yolov4-p6"]
]
YOLACT1 = ["Yolact", YOLACT_1, [
    "yolact-R50-FPN",
    "yolact-R101-FPN"]
]
YOLACT2 = ["Yolact++",YOLACT_2, [
    "yolact-plus-R50-FPN",
    "yolact-plus-R101-FPN"]
]
DETR = ["DETR", DETR_1, [
    "detr_r50",
    "detr_r50_dc5",
    "detr_r101"]
]
SOTR = ["SOTR", SOTR_1, [
    "sotr_R101",
    "sotr_R101_dcn"]
]
SOLO = ["SOLOv2", SOLO_1, [
    "solov2_R50_3x",
    "solov2_R101_3x"]
]

BMask_label = ["BMask", [MASK_2, CASK_1], []]

BMask = [None, MASK_2,
    ["bmask_rcnn_r50_1x",
    "bmask_rcnn_r101_1x"]
]
     
CBMask = [None, CASK_1,
    ["cascade_bmask_rcnn_r50_1x",  
    "cascade_bmask_rcnn_r101_1x"]
]
BCNet = ["F-BCNet", BCNET_1,
    ["fcos_imprv_R_101_FPN"]
]

In [12]:
def compare_models_line(MODEL_GROUPS, COLORS, LABELS,
                       DATASETS=[STYLIZED_COCO, STYLIZED_COCO_OBJECTS, STYLIZED_COCO_BACKGROUND], 
                       VERSION="1", BLEND_MODES=['pixel_space', 'feature_space'],
                       iou_type='segm', # bbox or segm
                       STATS=["AP", "APs", "APm", "APl"],
                       REL_ERROR=True,
                       COLUMN='stats'):

    plots = [] # collect all plots here
    for d_idx, DATASET in enumerate(DATASETS):
        
        # one plot per stat:
        for s_idx, stat in enumerate(STATS):

            # prepare a new figure for this statistic
            w,h = (260,220) 
            # enlarge depending on grid position (titles only on first/last plots)
            if COLUMN == 'stats':
                if s_idx == 0: h+= 20      
                if s_idx == len(STATS) - 1: h+=20
                if d_idx == len(DATASETS) - 1: w+=20
            else:
                if d_idx == 0: h+= 20      
                if s_idx == len(STATS) - 1: w+=20
                if d_idx == len(DATASETS) - 1: h+=20
            
            p = figure(x_range=(-0.05, 1.05), y_range=(-0.05,1.05), plot_width=w, plot_height=h,
                       x_axis_label="", toolbar_location=None, output_backend='svg', 
                       min_border_top=5, min_border_right=5)

            # fixed axis
            ALPHAS=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
            p.xaxis.ticker=FixedTicker(ticks=ALPHAS)
            p.yaxis.ticker=FixedTicker(ticks=ALPHAS)

            # add titles depending on grid position (titles only on first/last plots)
            dataset_name = [x.capitalize() for x in DATASET.split('_')]
            dataset_name = " ".join([dataset_name[0], dataset_name[-1]])

            d_title = Title(text=f"{dataset_name}", align="left")
            s_title = Title(text=f"{'r' if REL_ERROR else ''}{stat}{(' ('+ iou_type + ')') if iou_type == 'bbox' else ''}", align="left")

            if COLUMN == 'stats':
                if s_idx == 0:                 p.add_layout(d_title, "above")
                if s_idx == len(STATS) - 1:    p.xaxis.axis_label = 'Blending alpha'
                if d_idx == len(DATASETS) - 1: p.add_layout(s_title, "right")
            else:
                if d_idx == 0:                 p.add_layout(s_title, "above")
                if d_idx == len(DATASETS) - 1: p.xaxis.axis_label = 'Blending alpha'
                if s_idx == len(STATS) - 1:    p.add_layout(d_title, "right")

                
            # collect and plot data
            m_start,m_end = [],[]
            f_start,f_end = [],[]
            for MODELS, color in zip(MODEL_GROUPS, COLORS):
                
                if len(MODELS) == 0: continue

                # prepare mean dict for this model group
                avgs = {}
                for blend_mode in BLEND_MODES:
                    avgs[blend_mode] = []

                for model in MODELS:
                    for blend_mode in BLEND_MODES:

                        if iou_type == 'segm' and 'yolo' in model:
                            iou = 'bbox'
                        else:
                            iou = iou_type

                        y = SUMMARY[model][DATASET][VERSION][blend_mode][iou][stat]

                        if REL_ERROR:
                            # the uncorrupted coco dataset is alpha=0 in pixel space
                            coco_score = SUMMARY[model][DATASET][VERSION]['pixel_space'][iou][stat][0]
                            y = y / coco_score

                        # plot the model line
                        #p.line(ALPHAS, y, color=color, line_width=1, line_alpha=0.05)
                        #plot_augmented_line(p, x, y, color, [0,0], line_width=1, line_alpha=0.05)

                        # save y for mean calculation
                        avgs[blend_mode].append(y)

                # calculate mean and plot mean values
                for blend_mode in BLEND_MODES:
                    y = np.mean(avgs[blend_mode], axis=0)
                    alpha = 1 if blend_mode == 'feature_space' else 0.1
                    dash  = 'dashed' if 'bmask' in MODELS[0] else 'solid'
                    lw    = 2.5 if 'bmask' in MODELS[0] else 1.5
                    p.line(ALPHAS, y, color=color, line_width=lw, line_alpha=alpha, line_dash=dash)
                    #plot_augmented_line(p, x, y, color, [0,0], line_width=1.5, line_alpha=alpha)
                
                S = 'APs'
                D = STYLIZED_COCO_OBJECTS
                if DATASET == D and stat == S:
                    
                    m_start.extend([x[0] for x in avgs['feature_space']])
                    m_end.extend([x[-1] for x in avgs['feature_space']])
                    
                    f_start.append(np.mean([x[0] for x in avgs['feature_space']]))
                    f_end.append(np.mean([x[-1] for x in avgs['feature_space']]))
                    
            if DATASET == D and stat == S:
                print(f"Models: {np.mean(m_start):0.2f}+-{np.std(m_start):0.2f} -> {np.mean(m_end):0.2f}+-{np.std(m_end):0.2f}")
                print(f"Framew: {np.mean(f_start):0.2f}+-{np.std(f_start):0.2f} -> {np.mean(f_end):0.2f}+-{np.std(f_end):0.2f}")
            plots.append(p)

    # Legend as fake plot
    legend = figure(plot_width=800, plot_height=100, toolbar_location=None, output_backend='svg')
    legend.axis.visible, legend.outline_line_color, legend.grid.visible = False, None, False
    items  = []
    for c, l in zip(COLORS, LABELS):
        
        if l is None: continue
        if l == "BMask":
            c1,c2 = c
            r = legend.square_cross(0,0,0,0, color=c2, line_color=c1)
        else:
            r = legend.square(0,0,0,0,color=c)
        items.append((l,[r])) # legend items are tuples: ('str', [renderer])

    legend.add_layout(Legend(items=items, title='Frameworks:', location='left',
                             orientation="horizontal", glyph_height=30, glyph_width=30,
                             label_text_font_size='11px', label_standoff=-2, spacing = -2), 
                      "below")
    
    
    # plots are [[AP,APs,APm,APl],[AP,APs,APm,APl],[AP,APs,APm,APl]]
    # we group them by stats -> [[AP,AP,AP],[APs,APs,APs], ...] and apply the row layout per stat
    if COLUMN == 'stats':
        plots2 = [row(plots[i::len(STATS)]) for i in range(len(DATASETS)+1)]
    else:
        plots2 = [row(plots[0:4])]
        plots2 += [row(plots[4:8])]
        plots2 += [row(plots[8:])]
    
    grid = column([legend]+plots2)
    #grid = column(legend)
    show(grid)
    
    return grid # for saving


In [13]:
MODEL_GROUPS = []
COLORS = []
LABELS = []
#for l, c, mg in [YOLO, YOLACT1, YOLACT2, IN, LSJ, SLL, CASCADE, SOLO, SWIN_M, SWIN_C, DETR, SOTR]:
#for l, c, mg in [MASK, CASCADE, SWIN, YOLO, YOLACT1, YOLACT2, DETR, SOTR, SOLO]:
#for l, c, mg in [MASK, BMask_label, CASCADE, SWIN, YOLO, YOLACT1, YOLACT2, DETR, BCNet, SOTR, SOLO, BMask, CBMask]:
for l, c, mg in [MASK, BMask_label, CASCADE, SWIN,        YOLACT1, YOLACT2, DETR, BCNet, SOTR, SOLO, BMask, CBMask]:
    MODEL_GROUPS.append(mg)
    COLORS.append(c)
    LABELS.append(l)
    
print(len(MODEL_GROUPS))    

12


In [14]:
p = compare_models_line(MODEL_GROUPS, COLORS, LABELS, REL_ERROR=True, iou_type='segm', COLUMN='datasets')
#export_svg(p, filename=f"./test.svg")

Models: 0.36+-0.06 -> 0.32+-0.05
Framew: 0.40+-0.07 -> 0.34+-0.07


In [17]:
!which geckodriver

/usr/bin/geckodriver


## Compare Models in detail

In [ ]:
# some util functions
def plot_augmented_line(p, x, y, color, dash=[0,0], line_width=2, marker=None, start=None, end=None, line_alpha=1):

    # plot the line
    r = p.line(x, y, color=color, line_dash=dash, line_width=line_width, line_alpha=line_alpha)
    
    # marker, start and end have to be a tuple of (glyph function, size, color)
    if marker: marker[0](x,    y,     size=marker[1], color=marker[2])
    if start:   start[0](x[0], y[0],  size=start[1],  color=start[2], fill_color=None)
    if end:
        if marker: marker[0](x[-1],y[-1], size=marker[1], color=end[2])
        end[0](              x[-1],y[-1], size=end[1],    color=end[2], fill_color=None)
    
    return r

# because gridplots can't have a common legend, we create empty fake plots
def get_empty_plot(plot_width, plot_height=0, title=None):
    p = figure(plot_width=plot_width, plot_height=plot_height, toolbar_location=None, output_backend='svg', title=title)
    p.axis.visible, p.outline_line_color, p.grid.visible = False, None, False
    return p

# mark a plot if requested :)
def mark_plot(p):
    p.outline_line_width = 7
    p.outline_line_alpha = 0.3
    p.outline_line_color = "navy"

In [ ]:
def compare_datasets(MODEL, 
                     DATASETS=[STYLIZED_COCO, STYLIZED_COCO_OBJECTS, STYLIZED_COCO_BACKGROUND], 
                     VERSION='1',
                     BLEND_MODES=['pixel_space', 'feature_space'],
                     IOU_TYPES=['bbox', 'segm'],
                     STATS=["AP", "AP50", "AP75", "APs", "APm", "APl"],
                     MARKER=[], # list of stats strings, will mark the corresponding plots.
                     REL_ERROR=False, NCOLS=3):
    """
    This function plots AP, AP50, AP75, APs, APm and APl
    for a specific model and dataset(s) combination.
    
    It compares the effect of increasing corruption in 
    pixel and feature space on different datasets.
    
    returns: plot
    """
    
    #
    # We first define some general figure aesthetics
    #
    # color = blend mode
    colors  = {'pixel_space': '#1E88E5', 'feature_space': '#FFC107'}
    colors  = [colors[bm] for bm in  BLEND_MODES]

    # dash  = iou type
    dashes  = {'bbox': [4,4], 'segm':[0,0] }
    dashes  = [dashes[iou] for iou in IOU_TYPES]

    # datasets use different markers (glyphs)
    # function bc we need to call this per figure instance. TO-DO: do we, really?
    def get_markers(p, DATASETS): 
        markers = {'stylized_coco'           : [(p.circle,   10), (p.circle_dot,   20)],
                   'stylized_coco_objects'   : [(p.triangle, 10), (p.triangle_dot, 20)],
                   'stylized_coco_background': [(p.square,  8.5), (p.square_dot,   20)]}
        markers = [markers[d] for d in DATASETS]
        return markers
    
    #
    # DATA PLOTTING
    #
    # x values [0.0, ..., 1.0]
    x = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

    # collect all plots here
    plots = []
    
    # one plot per stat: 
    for stat in STATS:
        
        # link the plots by reusing existing axis ranges, else define new ones
        if plots: 
            x_range, y_range = plots[0].x_range, plots[0].y_range
        else:
            x_range = None
            y_range = (0, 1.1) if REL_ERROR else (0, 0.7)
        
        # prepare a new figure for this statistic
        p = figure(title=f"{'r' if REL_ERROR else ''}{stat}", x_range=x_range, y_range=y_range,
                   x_axis_label= "blending alpha", output_backend='svg')

        p.title.align = "center"
        p.xaxis.ticker=FixedTicker(ticks=x) # 0.1 steps
        p.yaxis.ticker=FixedTicker(ticks=x) # 0.1 steps
        if stat in MARKER: mark_plot(p)

        # plot data 
        for dataset, marker in zip(DATASETS, get_markers(p, DATASETS)):
            m1,s1 = marker[0] # (marker,    size)
            m2,s2 = marker[1] # (start/end, size)
            
            for blend_mode, color in zip(BLEND_MODES, colors):
                for iou_type, dash in zip(IOU_TYPES, dashes):
                    if iou_type not in SUMMARY[MODEL][dataset][VERSION][blend_mode].keys(): continue
                    
                    # the actual data for this statistic
                    y = SUMMARY[MODEL][dataset][VERSION][blend_mode][iou_type][stat]
                    if REL_ERROR:
                        # the uncorrupted coco dataset is alpha=0 in pixel space
                        coco_score = SUMMARY[MODEL][dataset][VERSION]['pixel_space'][iou_type][stat][0]
                        y = y / coco_score

                    plot_augmented_line(p, x, y, color, dash, 
                                        marker=(m1, s1, color), 
                                        start =(m2, s2, color),
                                        end   =(m2, s2, "#D81B60"))
        
        plots.append(p)
    
    #
    # LEGENDS
    #
    # we create a legend per blend mode
    legends = []
    for blend_mode, color in zip(BLEND_MODES, colors):
            
        items = [] # items will be grouped
        p = get_empty_plot(150)
        for iou_type, dash in zip(IOU_TYPES, dashes):
            r = plot_augmented_line(p, 0, 0, color, dash)
            items.append((iou_type, [r])) # legend items are tuples: ('str', [renderer])

        title = ' '.join(blend_mode.split('_'))
        p.add_layout(Legend(items=items, title=title, location='left', 
                            orientation="horizontal", glyph_height=30), "below")
        legends.append(p)
        
    # an a third legend to distinguish the datasets
    items = [] # items will be grouped
    p = get_empty_plot(300)
    for dataset, marker in zip(DATASETS, get_markers(p, DATASETS)):
        m1,s1 = marker[0]
        r = m1([0],[0], color='#888888')
        items.append((dataset.split('_')[-1],[r])) # legend items are tuples: ('str', [renderer])

    p.add_layout(Legend(items=items, title='stylized dataset:', location='left', 
                        orientation="horizontal", glyph_height=30, glyph_width=30), "below")
    legends.append(p)
     
    #
    # LAYOUT
    #
    #grid  = gridplot(plots, ncols=NCOLS, plot_width=300, plot_height=300, toolbar_location="right")
    grid  = gridplot(plots, ncols=NCOLS, plot_width=230, plot_height=250, toolbar_location="right")
    
    model = MODEL.strip('mask_rcnn_').strip("_3x").replace('_32x8d','')
    title = Div(text=f"<h3>Results for: {model}</h3>", align='center')

    #plot = column(title, grid, row(legends)) 
    print(f"{model} - {DATASETS}")
    #plot = column(grid, row(legends)) 
    plot = column(grid) 
    show(plot)

    return plot # for saving


In [ ]:
BASELINES_IN_PRE = [
    "mask_rcnn_R_50_C4_1x",
    "mask_rcnn_R_50_DC5_1x",
    "mask_rcnn_R_50_FPN_1x",
    "mask_rcnn_R_50_C4_3x",
    "mask_rcnn_R_50_DC5_3x",
    "mask_rcnn_R_50_FPN_3x",
    "mask_rcnn_R_101_C4_3x",
    "mask_rcnn_R_101_DC5_3x",
    "mask_rcnn_R_101_FPN_3x",
    "mask_rcnn_X_101_32x8d_FPN_3x"
]
BASELINES_NO_PRE = [
    "mask_rcnn_R_50_FPN_100ep_LSJ",
    "mask_rcnn_R_50_FPN_200ep_LSJ",
    "mask_rcnn_R_50_FPN_400ep_LSJ",
    "mask_rcnn_R_101_FPN_100ep_LSJ",
    "mask_rcnn_R_101_FPN_200ep_LSJ",
    "mask_rcnn_R_101_FPN_400ep_LSJ",
    "mask_rcnn_regnetx_4gf_dds_FPN_100ep_LSJ",
    "mask_rcnn_regnetx_4gf_dds_FPN_200ep_LSJ",
    "mask_rcnn_regnetx_4gf_dds_FPN_400ep_LSJ",
    "mask_rcnn_regnety_4gf_dds_FPN_100ep_LSJ",
    "mask_rcnn_regnety_4gf_dds_FPN_200ep_LSJ",
    "mask_rcnn_regnety_4gf_dds_FPN_400ep_LSJ"
]
OTHER_SETTINGS = [
    "cascade_mask_rcnn_R_50_FPN_1x",
    "cascade_mask_rcnn_R_50_FPN_3x",
    "mask_rcnn_R_50_FPN_1x_dconv_c3-c5",
    "mask_rcnn_R_50_FPN_3x_dconv_c3-c5",
    "panoptic_fpn_R_101_dconv_cascade_gn_3x",
    "cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv"
]
SELF_SUPERVISED = [
    "R_50_FPN_Random_6x",
    "R_50_FPN_Supervised_6x",
    "R_50_FPN_InstDis_1x",
    "R_50_FPN_InstDis_2x",
    "R_50_FPN_PIRL_1x",
    "R_50_FPN_PIRL_2x",
    "R_50_FPN_MoCo_v1_1x",
    "R_50_FPN_MoCo_v1_2x",
    "R_50_FPN_MoCo_v2_2x",
    "R_50_FPN_InfoMin_1x",
    "R_50_FPN_InfoMin_2x",
    "R_50_FPN_InfoMin_6x"
]
DETR =[
    "detr_r50",
    "detr_r50_dc5",
    "detr_r101"
]
YOLO = [
    "yolov3",
    "yolov4",
    "yolov4-csp",
    "yolov4-p5",
    "yolov4-p6"
]
SWIN = [
    "mask_rcnn_swin-T_1x",
    "mask_rcnn_swin-T_3x",
    "mask_rcnn_swin-S_3x",
    "cascade_mask_rcnn_swin-T_1x",
    "cascade_mask_rcnn_swin-T_3x",
    "cascade_mask_rcnn_swin-S_3x",
    "cascade_mask_rcnn_swin-B_3x"
]
SOTR = [
    "sotr_R101",
    "sotr_R101_dcn"
]
SOLO = [
    "solov2_R50_3x",
    "solov2_R101_3x"
]
YOLACT = [
    "yolact-R50-FPN",
    "yolact-R101-FPN",
    "yolact-plus-R50-FPN",
    "yolact-plus-R101-FPN"
]

In [ ]:
MODELS = []
MODELS.extend(BASELINES_IN_PRE)
MODELS.extend(BASELINES_NO_PRE)
#MODELS.extend(SELF_SUPERVISED)
MODELS.extend(OTHER_SETTINGS)
MODELS.extend(DETR_PANOPTIC)
#MODELS.extend(YOLO)
MODELS.extend(SWIN)
MODELS.extend(SOTR)
MODELS.extend(SOLO)
MODELS.extend(YOLACT)
print(len(MODELS))
#MODELS = ["mask_rcnn_R_50_C4_1x","mask_rcnn_R_50_C4_1x"]

46


In [ ]:
for model in MODELS:
    compare_datasets(model, 
                     DATASETS=[STYLIZED_COCO], 
                     STATS=["AP", "APs", "APm", "APl"],
                     REL_ERROR=True, NCOLS=6)

R_50_C4_1 - ['stylized_coco']


R_50_DC5_1 - ['stylized_coco']


R_50_FPN_1 - ['stylized_coco']


R_50_C4 - ['stylized_coco']


R_50_DC5 - ['stylized_coco']


...

R_101_C4 - ['stylized_coco']


R_101_DC5 - ['stylized_coco']


R_101_FPN - ['stylized_coco']


X_101_FPN - ['stylized_coco']


KeyboardInterrupt: 